# Data Exploration and Visualization of Cosmic Strings in CMB
This notebook explores the Cosmic Microwave Background (CMB) data to identify potential cosmic string candidates.

Initial exploration of the CMB data using Healpy and Matplotlib.

In [ ]:
# Install required packages
# pip freeze > requirements.txt
!pip install -r requirements.txt

First exploring the COM_CMB_IQU-smica_2048_R3.00_full.fits file to understand its structure and contents.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.constants.codata2014 import alpha
from scipy.constants import h, c, k
import healpy as hp

# Planck function (spectral radiance)
def planck(f, T):
    x = h * f / (k * T)
    return (2 * h * f**3 / c**2) / np.expm1(x)


# Frequency range (Hz)
frequencies = np.linspace(1e10, 1e12, 1000)  # 10 GHz to 1000 GHz
temperature = 2.725  # CMB temperature in Kelvin

# Compute spectral radiance
radiance = planck(frequencies, temperature)

# Plot spectrum
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, radiance, color='darkblue', label=f'T = {temperature} K')
plt.title('CMB Blackbody Spectrum at T = 2.725 K')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Spectral Radiance B(ν) [W·sr⁻¹·m⁻²·Hz⁻¹]')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import healpy as hp
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from skimage import exposure

# --- Load the CMB temperature map ---
cmb_map = hp.read_map('data/COM_CMB_IQU-smica_2048_R3.00_full.fits', field=0)

# --- Optional: Apply Galactic/common mask ---
load_mask = False

if load_mask:
    cmb_mask = hp.read_map('data/COM_Mask_CMB-common-Mask-Int_2048_R3.00.fits', field=0)

    # Sanity check: Ensure matching resolution
    assert hp.get_nside(cmb_map) == hp.get_nside(cmb_mask), "NSIDE mismatch between map and mask!"

    # Apply the mask (set bad pixels to 0)
    cmb_map = cmb_map * cmb_mask

# --- Convert to 2D Mollweide projection ---
img = hp.mollview(
    cmb_map,
    return_projected_map=True,
    nest=False,
    title='CMB Temperature Map',
    cmap='inferno',
    xsize=2000
)

# --- Optional enhancement: Histogram equalization for better contrast ---
img_eq = exposure.equalize_hist(img)

In [ ]:
# --- Apply Sobel edge detection to reveal structure ---
edges = ndimage.sobel(img_eq)

# --- Display the edges ---
plt.figure(figsize=(10, 5))
plt.imshow(edges, cmap='gray')
plt.title("Edge Detection on Equalized CMB Map")
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

# --- Step 1: Load or define your image (already done)
# img = ...

# --- Step 2: Force-clean the CMB image ---
# Replace NaNs, -inf, +inf with zeros (or median)
img_cleaned = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)

# --- Step 3: Edge detection ---
edges = ndimage.sobel(img_cleaned)

# --- Step 4: Print CLEANED ranges ---
print("Cleaned CMB image range:", np.min(img_cleaned), "to", np.max(img_cleaned))
print("Edge image range:", np.min(edges), "to", np.max(edges))

# --- Step 5: Normalize for plotting ---
def normalize(arr):
    arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
    return (arr - np.min(arr)) / (np.ptp(arr) + 1e-8)

img_norm = normalize(img_cleaned)
edges_norm = normalize(edges)

# Optional: Threshold edges for clarity
edges_thresh = np.where(edges_norm > 0.3, 1.0, 0.0)

# --- Step 6: Plot overlay ---
plt.figure(figsize=(14, 6))
plt.imshow(img_norm, cmap='coolwarm',alpha=0.9, origin='lower', aspect='auto')
plt.imshow(edges_thresh, cmap='YlOrBr', alpha=0.9, origin='lower', aspect='auto')
plt.colorbar(label='Intensity')
plt.title("CMB with Edge Overlay (cleaned)")
plt.axis('off')
plt.tight_layout()
plt.savefig("cmb_edge_overlay_cleaned.png", dpi=300)
plt.show()


In [ ]:
# Nonlinear exaggeration
edges_exaggerated = edges**0.8  # or try 2.0


plt.figure(figsize=(30, 15))
plt.imshow(edges_exaggerated, cmap='turbo')
plt.title('Possible Early Universal Topology Signatures (CMB Edge Detection)')
plt.colorbar()
plt.show()

In [ ]:
sharp_edges = edges + 0.7 * edges  # amplify edge intensity


plt.figure(figsize=(30, 15))
plt.imshow(sharp_edges, cmap='inferno')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()


In [ ]:
# Alter this code block
# Apply edge detection (e.g., Sobel)edges = ndimage.sobel(img)
# Plot the edges detected in the CMB map
# Apply Sobel in x and y directions
dx = ndimage.sobel(img, axis=0) ** 0.5
dy = ndimage.sobel(img, axis=1) ** 0.5

# Gradient magnitude
edges = np.hypot(dx, dy)  # Same as sqrt(dx**2 + dy**2)

plt.figure(figsize=(30, 15))
plt.imshow(edges, cmap='inferno')
plt.title('Possible Cosmic String Candidates (Edges in CMB)')
plt.colorbar()
plt.show()


In [ ]:
frequencies = np.linspace(10e9, 1000e9, 1000)  # 10 GHz to 1000 GHz

T_mean = 2.725
delta_T = 100e-6  # 100 µK typical fluctuation

plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean + delta_T), '--', label="+100 µK", color='green', alpha=0.8)
plt.plot(frequencies / 1e9, planck(frequencies, T_mean - delta_T), '--', label="-100 µK", color='red', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Show the CMB spectrum with +100 µK fluctuations
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean + delta_T), '--', label="+100 µK", color='green', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Show the CMB spectrum with -100 µK fluctuations
plt.figure(figsize=(10, 6))
plt.plot(frequencies / 1e9, planck(frequencies, T_mean), label="T = 2.725 K", color='blue')
plt.plot(frequencies / 1e9, planck(frequencies, T_mean - delta_T), '--', label="-100 µK", color='red', alpha=0.8)
plt.xlabel("Frequency (GHz)")
plt.ylabel("Spectral Radiance (W·sr⁻¹·m⁻²·Hz⁻¹)")
plt.title("CMB Spectrum with Fluctuation Bounds (±100 µK)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
cl = hp.anafast(cmb_map)
plt.plot(cl[:200])
plt.title("Low-ℓ Multipoles – Texture Signature Region")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from matplotlib.ticker import FuncFormatter

# --- Compute power spectrum from CMB map ---
cl = hp.anafast(cmb_map)         # Power spectrum
ell = np.arange(len(cl))         # Multipole indices

# --- Begin Plot ---
plt.figure(figsize=(12, 6))

# Plot measured CMB power spectrum (excluding monopole)
plt.loglog(ell[1:1500], cl[1:1500], label='CMB Power Spectrum', color='blue')

# Plot theoretical scale-invariant reference curve
ref_value = np.mean(cl[5:15])
theory_curve = ref_value * (ell[5] * (ell[5] + 1)) / (ell[1:1500] * (ell[1:1500] + 1))
plt.loglog(ell[1:1500], theory_curve, 'r--', label='Scale-invariant (∝ 1/ℓ(ℓ+1))')

# --- Highlight Regions ---
plt.axvspan(2, 30, color='gray', alpha=0.15, label='Sachs-Wolfe Region')
plt.axvspan(50, 200, color='orange', alpha=0.08, label='Transition Region')
plt.axvspan(200, 1500, color='green', alpha=0.08, label='Acoustic Peaks Region')

# --- Annotations (repositioned to avoid overlap) ---
plt.text(7, 2e-10, 'Sachs-Wolfe\nSuper-horizon modes', fontsize=10, color='black')
plt.text(60, 5e-20, 'Transition:\nEarly Oscillations &\n Projection Effects', fontsize=10, color='black')
plt.text(300, 5e-12, 'Acoustic Peaks:\nPhoton-Baryon Oscillations', fontsize=10, color='black')

# 1st Peak marker
plt.axvline(220, color='black', linestyle='--', alpha=0.5)
plt.text(235, 1e-10, '1st Peak\n(~ℓ=220)', fontsize=9, color='black')

# --- Labels, Legend, Grid ---
plt.title("CMB Angular Power Spectrum (Log Scale)")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.2)

# --- Format axes using human-readable numbers ---
def human_format(x, pos):
    if x >= 1:
        return f"{int(x)}"
    else:
        return f"{x:.1g}"

ax = plt.gca()
ax.xaxis.set_major_formatter(FuncFormatter(human_format))
ax.yaxis.set_major_formatter(FuncFormatter(human_format))

plt.tight_layout()
plt.show()


In [ ]:
# Plot the power spectrum (ℓ = 0 to 199)
ell = np.arange(len(cl))  # Define ell as the array of multipole indices
plt.figure(figsize=(9, 5))
plt.plot(ell[:250], cl[:250], label='CMB Power')

# Overlay topological defect regions
plt.axvspan(2, 10, color='blue', alpha=0.3, label='Textures (ℓ ≈ 2–10)')
plt.axvspan(2, 5, color='red', alpha=0.3, label='Domain Walls (ℓ ≈ 2–5)')
plt.axvspan(50, 200, color='green', alpha=0.2, label='Cosmic Strings (ℓ ≈ 50–200)')
plt.axvspan(2, 3, color='orange', alpha=0.4, label='Monopoles (ℓ ≈ 2–3)')

# Labels and grid
plt.title("CMB Multipole Spectrum with Topological Defect Regions")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot the power spectrum (ℓ = 0 to 199)
ell = np.arange(len(cl))  # Define ell as the array of multipole indices
plt.figure(figsize=(9, 5))
plt.plot(ell[:50], cl[:50], label='CMB Power')

# Overlay topological defect regions
plt.axvspan(2, 10, color='blue', alpha=0.3, label='Textures (ℓ ≈ 2–10)')
plt.axvspan(2, 5, color='red', alpha=0.3, label='Domain Walls (ℓ ≈ 2–5)')
plt.axvspan(2, 3, color='orange', alpha=0.4, label='Monopoles (ℓ ≈ 2–3)')

# Labels and grid
plt.title("CMB Multipole Spectrum with Topological Defect Regions")
plt.xlabel("Multipole ℓ [ℓ: inverse angular scale]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Use same `cl` from previous block
ell = np.arange(1, 200)  # Skip ℓ = 0 to avoid divide-by-zero
theta_deg = 180 / ell    # Angular scale in degrees

# Trim cl accordingly
cl_trim = cl[1:200]

# Plot C_ell vs angular scale θ
plt.plot(theta_deg, cl_trim)
plt.title("CMB Power Spectrum vs Angular Scale")
plt.xlabel("Angular Scale θ [degrees]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.gca().invert_xaxis()  # Larger angular scales on the left
plt.show()

In [ ]:
# ℓ = 1 to 199 (skip ℓ = 0 to avoid division by zero)
theta_deg = 180 / ell
cl_trim = cl[1:200]

# Plot power vs angular scale
plt.figure(figsize=(9, 5))
plt.plot(theta_deg, cl_trim, label='CMB Power')

# Overlay topological defect regions in degrees
plt.axvspan(18, 90, color='blue', alpha=0.3, label='Textures (θ ≈ 18° to 90°)')
plt.axvspan(36, 90, color='red', alpha=0.3, label='Domain Walls (θ ≈ 36° to 90°)')
plt.axvspan(1, 4, color='green', alpha=0.2, label='Cosmic Strings (θ ≈ 1° to 4°)')
plt.axvspan(60, 90, color='orange', alpha=0.4, label='Monopoles (θ ≈ 60° to 90°)')

# Labels and formatting
plt.title("CMB Power vs Angular Scale with Topological Defect Regions")
plt.xlabel("Angular Scale θ [degrees]")
plt.ylabel("C_ℓ [Power: variance of temperature fluctuations]")
plt.grid()
plt.gca().invert_xaxis()  # Large scales (low ℓ) on left
plt.legend()
plt.tight_layout()
plt.show()


Example of exaggerated cosmic strings

In [ ]:
from skimage import draw  # Changed import to use skimage.draw instead of matplotlib.pyplot.draw

def simulate_cosmic_strings(map_size, num_strings):
    map_data = np.zeros((map_size, map_size))
    for _ in range(num_strings):
        x = np.random.randint(0, map_size)
        y = np.random.randint(0, map_size)
        dx = np.random.randint(-map_size//4, map_size//4)
        dy = np.random.randint(-map_size//4, map_size//4)
        rr, cc = draw.line(x, y, x+dx, y+dy)  # Now using skimage.draw.line
        map_data[rr % map_size, cc % map_size] += np.random.choice([-1, 1]) * 100e-6  # ~100 µK jump
    return map_data

# Simulate and plot
map_size = 1000  # Reduced size for better visualization
num_strings = 10
cosmic_map = simulate_cosmic_strings(map_size, num_strings)

plt.figure(figsize=(10, 10))
plt.imshow(cosmic_map, cmap='RdBu_r')
plt.colorbar(label='Temperature fluctuation (K)')
plt.title('Simulated Cosmic Strings')
plt.show()


## Exploring the data with Machine Learning Methods

In this section, the application of various machine learning techniques to analyze the CMB data and identify potential patterns or anomalies that might be associated with cosmic strings or other phenomena.

Ensure the data and necessary libraries are loaded (just means sections below are not reliant on sections above.)


In [ ]:
# Import necessary ML libraries
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from scipy import stats

# Make sure we have the CMB map loaded
# If not already loaded, load it again
if 'cmb_map' not in locals():
    cmb_map = hp.read_map('data/COM_CMB_IQU-smica_2048_R3.00_full.fits', field=0)
    nside = hp.get_nside(cmb_map)
    npix = hp.nside2npix(nside)
    img = hp.mollview(cmb_map, return_projected_map=True, nest=False, title='CMB Temperature Map', cmap='inferno', xsize=2000, hold=True)
    img_cleaned = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)

print(f"CMB map shape: {cmb_map.shape}")
print(f"Projected image shape: {img_cleaned.shape}")


### 1. Feature Extraction and Dimensionality Reduction

First, extracting features from the CMB data and reducing dimensionality to visualize patterns.


### For Single Cluster

In [ ]:
# For ML analysis, we'll work with the 2D projected map (img_cleaned)
# Let's extract patches from the image to use as features

def extract_patches(image, patch_size=16, stride=8):
    """Extract patches from a 2D image with a given stride."""
    patches = []
    positions = []
    h, w = image.shape

    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            # Skip patches with NaN or inf values
            patch = image[i:i+patch_size, j:j+patch_size]
            if np.isfinite(patch).all() and not np.isnan(patch).any():
                # Flatten the patch to a 1D array
                patches.append(patch.flatten())
                positions.append((i, j))

    return np.array(patches), positions

# Extract patches from the cleaned image
# patch_size = 4 , stride = 2	Very localized features, fast	No texture, easy to overfit noise
# patch_size = 8 , stride = 4	Ultra-local	Very sensitive to noise; good for edge detection
# patch_size = 16, stride = 8	Small patches, overlapping	Dense coverage, good for texture analysis
# patch_size = 32, stride = 32	Large, non-overlapping	Coarse, fewer samples but high information per patch
# patch_size = 64, stride = 16	Large and overlapping	Heavy computation; useful for large-scale pattern mining
#
#
# Try three experimental combinations:
# Test	Patch Size	Stride	Use Case
# A	    16	        8	    Balanced: local structure + manageable data
# B	    32	        16	    Large structures, good for global coherence
# C	    8	        4	    Detect sharp features (e.g., cosmic strings, shocks)
patch_size = 8
stride = 4
patches, positions = extract_patches(img_cleaned, patch_size, stride)

print(f"Extracted {len(patches)} patches of size {patch_size}x{patch_size}")

# Standardize the patches
scaler = StandardScaler()
patches_scaled = scaler.fit_transform(patches)

# Fit PCA with all components first to inspect explained variance
pca_full = PCA()
pca_full.fit(patches_scaled)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Define your target variance threshold (e.g., 95%)
#     target_variance: A pre-defined threshold (e.g., 0.95 for 95%) indicating how much of the
#     original data's variability PCA should preserve.
#
#     Purpose: Balances dimensionality reduction against information loss.
target_variance = 0.95

# Find the number of components that meet or exceed this threshold
n_components = np.argmax(cumulative_variance >= target_variance) + 1
print(f"Number of components to retain {target_variance*100:.0f}% variance: {n_components}")


# Apply PCA again with the optimal number of components
pca = PCA(n_components=n_components)
patches_pca = pca.fit_transform(patches_scaled)

# Reconstruct data with selected components
patches_reduced = pca.fit_transform(patches_scaled)
patches_reconstructed = pca.inverse_transform(patches_reduced)

# Calculate mean squared error
mse = np.mean((patches_scaled - patches_reconstructed) ** 2)
print(f"Reconstruction MSE: {mse:.2e}")

# Optional: plot the explained variance curve with a line at the threshold
plt.figure(figsize=(10, 5))
plt.plot(cumulative_variance, label='Cumulative Explained Variance')
plt.axhline(y=target_variance, color='r', linestyle='--', label=f'{target_variance*100:.0f}% Variance')
plt.axvline(x=n_components - 1, color='g', linestyle='--', label=f'{n_components} Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Explained Variance')
plt.legend()
plt.grid(True)
plt.show()


# Apply t-SNE for visualization
# t-Distributed Stochastic Neighbor Embedding
tsne = TSNE(n_components=2, random_state=42)
patches_tsne = tsne.fit_transform(patches_pca)

plt.figure(figsize=(10, 8))
plt.scatter(patches_tsne[:, 0], patches_tsne[:, 1], alpha=0.5, s=5)
plt.title('t-SNE Visualization of CMB Patches')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar()
plt.show()


Is This MSE Value Good?

Typical MSE Range for CMB Data:

Excellent Reconstruction: MSE < 1e-02 (near-perfect preservation of CMB features)

Good Reconstruction: MSE ~1e-02 to 1e-01 (retains most cosmological signals)

Moderate Reconstruction: MSE ~1e-01 to 5e-01 (some loss of small-scale fluctuations)

Poor Reconstruction: MSE > 5e-01 (significant smoothing/feature loss)

Aim for MSE < 1e-02 to ensure topology-sensitive features survive dimensionality reduction.

In [ ]:
if mse < 1e-02:
    print(f"Reconstruction MSE: {mse:.2e} is excellent for CMB data")
elif mse < 1e-01:
    print(f"Reconstruction MSE: {mse:.2e} is good for CMB data")
elif mse < 5e-01:
    print(f"Reconstruction MSE: {mse:.2e} is moderate for CMB data")
else:
    print(f"Reconstruction MSE: {mse:.2e} is not good for CMB data")

### For Combined Clusters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# --- PATCH EXTRACTION FUNCTION ---
def extract_patches(image, patch_size=16, stride=8):
    """Extract patches from a 2D image with a given stride."""
    patches = []
    positions = []
    h, w = image.shape

    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            patch = image[i:i+patch_size, j:j+patch_size]
            if np.isfinite(patch).all() and not np.isnan(patch).any():
                patches.append(patch.flatten())
                positions.append((i, j))

    return np.array(patches), positions

# --- PARAMETERS ---
patch_sizes = [8, 16, 32]  # Multi-scale patch sizes
stride = 4
target_variance = 0.95

# --- COLLECT ALL FEATURES ---
all_features = []
n_components_dict = {}
explained_variances = {}
mse_dict = {}

pca_models = {}
scalers = {}


for patch_size in patch_sizes:
    print(f"\n🔍 Processing patch size: {patch_size}x{patch_size}")

    # Extract patches
    patches, _ = extract_patches(img_cleaned, patch_size, stride)
    print(f"Extracted {len(patches)} patches of size {patch_size}x{patch_size}")

    # Standardize
    scaler = StandardScaler()
    patches_scaled = scaler.fit_transform(patches)

    # PCA to inspect variance
    pca_full = PCA()
    pca_full.fit(patches_scaled)
    cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

    # Determine components needed
    n_components = np.argmax(cumulative_variance >= target_variance) + 1
    n_components_dict[patch_size] = n_components
    explained_variances[patch_size] = cumulative_variance

    print(f"✅ {n_components} components retain {target_variance*100:.0f}% variance")

    # Apply PCA with optimal components
    pca = PCA(n_components=n_components)
    patches_pca = pca.fit_transform(patches_scaled)

    # Reconstruct from PCA
    patches_reconstructed = pca.inverse_transform(patches_pca)

    # Compute reconstruction MSE
    mse = np.mean((patches_scaled - patches_reconstructed) ** 2)
    mse_dict[patch_size] = mse
    print(f"📉 Reconstruction MSE for {patch_size}x{patch_size} patches: {mse:.2e}")

    # Store PCA and scaler for later analysis
    pca_models[patch_size] = pca
    scalers[patch_size] = scaler

    # Save features
    all_features.append(patches_pca)


# --- COMBINE MULTISCALE FEATURES ---
# --- Find common patch count across all scales ---
min_len = min(f.shape[0] for f in all_features)
print(f"\n🔧 Truncating all features to {min_len} samples for alignment")

# Truncate all feature arrays to the minimum number of patches
all_features_trimmed = [f[:min_len] for f in all_features]

# Concatenate safely
combined_features = np.concatenate(all_features_trimmed, axis=1)
print(f"🔗 Combined feature shape: {combined_features.shape}")


# --- OPTIONAL: Plot explained variance curves for each scale ---
plt.figure(figsize=(10, 6))
for size in patch_sizes:
    plt.plot(explained_variances[size], label=f'{size}x{size}')
plt.axhline(y=target_variance, color='r', linestyle='--', label='Target Variance')
plt.title('Cumulative Explained Variance per Patch Size')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.legend()
plt.grid(True)
plt.show()

# --- APPLY t-SNE ON COMBINED FEATURES ---
print("\n🎯 Applying t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate='auto')
features_tsne = tsne.fit_transform(combined_features)

# --- PLOT T-SNE ---
plt.figure(figsize=(10, 8))
plt.scatter(features_tsne[:, 0], features_tsne[:, 1], alpha=0.5, s=5)
plt.title('t-SNE of Multi-Scale PCA-Reduced CMB Patches')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True)
plt.show()

# --- PRINT MSE SUMMARY ---
print("\n📊 MSE Summary by Patch Size:")
for size in patch_sizes:
    print(f"  - {size}x{size}: MSE = {mse_dict[size]:.2e}")


Is This MSE Value Good?

Typical MSE Range for CMB Data:

Excellent Reconstruction: MSE < 1e-02 (near-perfect preservation of CMB features)

Good Reconstruction: MSE ~1e-02 to 1e-01 (retains most cosmological signals)

Moderate Reconstruction: MSE ~1e-01 to 5e-01 (some loss of small-scale fluctuations)

Poor Reconstruction: MSE > 5e-01 (significant smoothing/feature loss)

Aim for MSE < 1e-02 to ensure topology-sensitive features survive dimensionality reduction.

In [ ]:
# --- Evaluate Reconstruction Quality ---
print("📌 MSE Quality Assessment per Patch Size:")
for size in patch_sizes:
    mse = mse_dict[size]
    if mse < 1e-02:
        print(f"  ✅ {size}x{size}: Reconstruction MSE = {mse:.2e} is **excellent** for CMB data")
    elif mse < 1e-01:
        print(f"  👍 {size}x{size}: Reconstruction MSE = {mse:.2e} is **good** for CMB data")
    elif mse < 5e-01:
        print(f"  ⚠️ {size}x{size}: Reconstruction MSE = {mse:.2e} is **moderate** for CMB data")
    else:
        print(f"  ❌ {size}x{size}: Reconstruction MSE = {mse:.2e} is **not good** for CMB data")


### 2. Unsupervised Learning: Clustering Analysis

Now we'll apply clustering algorithms to identify groups of similar patterns in the CMB data.


#### Dynamic Optimization of Clustering Parameters

To find the optimal clustering of the CMB data, a dynamic approach is used that:

1. Tries different numbers of clusters (from 3 to 50)
2. For each number of clusters, runs K-means multiple times with different random initializations
3. Calculates the silhouette score for each clustering attempt
4. Selects the clustering with the highest silhouette score

This approach helps find the best clustering configuration automatically, rather than manually tuning parameters. The silhouette score measures how well-separated the clusters are, with higher scores indicating better-defined clusters.


1. Expected Silhouette Score Range for CMB Data

    Good clustering:

    0.5 - 1.0 → Strong evidence of cluster structure (rare for CMB unless studying clear anomalies like cold spots or non-Gaussian features).

    0.3 - 0.5 → Reasonable separation (may indicate subtle non-Gaussianities or foreground contamination).

    Ambiguous clustering:

    0.1 - 0.3 → Weak structure (common for Gaussian CMB fluctuations; clusters may be artificial).

    No meaningful clusters:

    ≤ 0.1 or negative → Likely noise or overfitting (common if forcing clusters on Gaussian random fields).

Key Insight:
CMB is mostly Gaussian, so high silhouette scores are unexpected unless you're targeting specific anomalies or foregrounds. A score of 0.2-0.4 might be the realistic upper limit for most analyses.


### For Single Cluster

In [ ]:
# Function to run K-means clustering multiple times and find the best silhouette score
def find_best_kmeans(data, min_clusters=2, max_clusters=10, n_attempts=10):
    """
    Run K-means clustering multiple times with different parameters to find the best silhouette score.

    Parameters:
    -----------
    data : array-like
        The data to cluster
    min_clusters : int
        Minimum number of clusters to try
    max_clusters : int
        Maximum number of clusters to try
    n_attempts : int
        Number of random initializations to try for each number of clusters

    Returns:
    --------
    best_kmeans : KMeans
        The best KMeans model
    best_labels : array
        The cluster labels from the best model
    best_n_clusters : int
        The number of clusters in the best model
    best_score : float
        The silhouette score of the best model
    """
    best_score = -1
    current_best_kmeans = -100
    best_kmeans = None
    best_labels = None
    best_n_clusters = 0
    break_score = 0

    # Try different numbers of clusters
    for n_clusters in range(min_clusters, max_clusters + 1):
        print(f"Trying {n_clusters} clusters...")
        # Try multiple random initializations for each number of clusters
        for attempt in range(n_attempts):
            # Initialize and fit KMeans
            kmeans = KMeans(n_clusters=n_clusters, random_state=attempt)
            labels = kmeans.fit_predict(data)
            # Calculate silhouette score
            score = silhouette_score(data, labels)
            current_best_kmeans = score
            print(f"  Attempt {attempt+1}/{n_attempts}: Silhouette Score = {score:.9f}")

            # Update best model if this one is better
            if score > best_score:
                best_score = score
                best_kmeans = kmeans
                best_labels = labels
                best_n_clusters = n_clusters
                print(f"  New best score: {best_score:.9f} with {best_n_clusters} clusters")
            if score <= 0.1:
                print(f"  No meaningful clusters found for {n_clusters} clusters")
                break
            if score <= best_score-0.05:
                print(f"  No improvement in silhouette score for {n_clusters} clusters")
                break

    print(f"\nBest clustering: {best_n_clusters} clusters with silhouette score {best_score:.9f}")
    return best_kmeans, best_labels, best_n_clusters, best_score

# Apply K-means clustering with multiple attempts to find the best silhouette score
min_clusters = 3
max_clusters = 10
n_attempts = 5
print(f"Finding best K-means clustering (trying {min_clusters}-{max_clusters} clusters, {n_attempts} attempts each)...")
best_kmeans, cluster_labels, best_n_clusters, best_silhouette = find_best_kmeans(
    patches_pca, min_clusters=min_clusters, max_clusters=max_clusters, n_attempts=n_attempts
)


print(f"t-SNE shape: {patches_tsne.shape}")
print(f"Cluster labels shape: {cluster_labels.shape}")


In [ ]:
patches_tsne_clustered = patches_tsne[:len(cluster_labels)]

# Visualize clusters in t-SNE space
plt.figure(figsize=(12, 10))
scatter = plt.scatter(patches_tsne_clustered[:, 0], patches_tsne_clustered[:, 1],
                      c=cluster_labels, cmap='viridis', alpha=0.7, s=10)

plt.colorbar(scatter, label='Cluster')
plt.title(f'K-means Clustering (k={best_n_clusters}, Silhouette={best_silhouette:.3f})')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()
# Calculate silhouette score to evaluate clustering quality
#     Higher Silhouette Score → Better clustering quality
#     (points are correctly assigned to tight, well-separated clusters).
#
#     Lower Silhouette Score → Worse clustering quality
#     (points may be misassigned or clusters overlap).
# 0.2-0.4 would be a good score for CMB data due to the nature of it
print(f"Best Silhouette Score: {best_silhouette:.3f} with {best_n_clusters} clusters")

In [ ]:
# Visualize cluster centers in image space
plt.figure(figsize=(15, 3))
for i in range(best_n_clusters):
    plt.subplot(1, best_n_clusters, i+1)
    # Get the center of the cluster in original space
    center = pca.inverse_transform(best_kmeans.cluster_centers_[i])
    center = scaler.inverse_transform([center])[0]
    # Reshape to patch size
    center = center.reshape(patch_size, patch_size)
    plt.imshow(center, cmap='inferno')
    plt.title(f'Cluster {i}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Map clusters back to the original image
cluster_map = np.zeros(img_cleaned.shape)
cluster_count = np.zeros(img_cleaned.shape)

for (i, j), label in zip(positions, cluster_labels):
    cluster_map[i:i+patch_size, j:j+patch_size] += label
    cluster_count[i:i+patch_size, j:j+patch_size] += 1

# Average the cluster labels where patches overlap
mask = cluster_count > 0
cluster_map[mask] /= cluster_count[mask]

plt.figure(figsize=(15, 10))
plt.imshow(cluster_map, cmap='viridis')
plt.title(f'Cluster Map of CMB Data (k={best_n_clusters}, Silhouette={best_silhouette:.3f})')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
# Check variance retention
# Aim for ≥95% for CMB (unlike images, where 80-90% may suffice).
print(f"Variance retained: {np.sum(pca.explained_variance_ratio_):.2%}")

## For Combined Clusters

In [ ]:
# --- PREP: Store clustering results ---
best_kmeans_models = {}
cluster_labels_dict = {}
retained_variance_dict = {}

# Choose patch sizes for fusion
selected_patch_sizes = patch_sizes  # example: use 4x4 and 16x16

# Apply silhouette-optimized KMeans per selected patch size
for size in selected_patch_sizes:
    print(f"\n🔍 Running KMeans with silhouette scoring on {size}x{size} patches")

    features = all_features[patch_sizes.index(size)][:min_len]  # truncate to match others
    best_kmeans, labels, n_clusters, silhouette = find_best_kmeans(
        features, min_clusters=3, max_clusters=5, n_attempts=3
    )

    best_kmeans_models[size] = best_kmeans
    cluster_labels_dict[size] = labels
    print(f"✅ Best k={n_clusters} with silhouette score={silhouette:.3f}")

# --- COMBINE MULTISCALE LABELS INTO META-FEATURE VECTOR ---
print("\n🔗 Building meta-feature vector from cluster labels")
meta_features = np.column_stack([cluster_labels_dict[size] for size in selected_patch_sizes])
print(f"Meta-feature shape: {meta_features.shape}")

# --- APPLY ANOMALY DETECTION TO META-FEATURE VECTOR ---
print("\n🌌 Running Isolation Forest on cluster fusion features")
iso = IsolationForest(contamination=0.05, random_state=42)
anomaly_labels = iso.fit_predict(meta_features)

# Convert to anomaly score (1 = normal, -1 = anomaly → flip)
anomaly_scores = -1 * anomaly_labels
print(f"Anomaly scores: {np.unique(anomaly_scores, return_counts=True)}")


In [ ]:
# --- VISUALIZE IN T-SNE SPACE ---
plt.figure(figsize=(12, 10))
plt.scatter(features_tsne[:len(anomaly_scores), 0], features_tsne[:len(anomaly_scores), 1],
            c=anomaly_scores, cmap='coolwarm', alpha=0.7, s=10)
plt.title('Anomaly Detection from Multi-Scale Cluster Fusion')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.colorbar(label='Anomaly Score')
plt.grid(True)
plt.show()


In [ ]:
# Apply K-means clustering with multiple attempts to find the best silhouette score
min_clusters = 3
max_clusters = 10
n_attempts = 5

print(f"\n🔎 Finding best K-means clustering on multi-scale features ({min_clusters}-{max_clusters} clusters, {n_attempts} attempts each)...")

# Use the multi-scale features for clustering
best_kmeans, cluster_labels, best_n_clusters, best_silhouette = find_best_kmeans(
    combined_features, min_clusters=min_clusters, max_clusters=max_clusters, n_attempts=n_attempts
)

# Visualize clusters in t-SNE space (already computed from combined_features)
plt.figure(figsize=(12, 10))
scatter = plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7, s=10)
plt.colorbar(scatter, label='Cluster')
plt.title(f'K-means Clustering on Multi-Scale Features (k={best_n_clusters}, Silhouette={best_silhouette:.3f})')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

# --- Track PCA variance retained ---
n_components_used = n_components_dict[size]
retained_variance = explained_variances[size][n_components_used - 1]
retained_variance_dict[size] = retained_variance

# Report silhouette quality (typical good range is 0.2–0.4 for noisy CMB)
print(f"📈 Best Silhouette Score: {best_silhouette:.3f} | {best_n_clusters} clusters | Variance Retained={retained_variance:.3f}" )


#### Alternate Method

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE
from scipy.stats import entropy

# --- PREP: Store clustering results ---
best_kmeans_models = {}
cluster_labels_dict = {}
silhouette_scores_per_size = {}
entropy_scores = {}

# --- CHOOSE PATCH SIZES FOR META-FUSION ---
selected_patch_sizes = patch_sizes  # or e.g., [4, 8, 16]

# --- SILHOUETTE-OPTIMIZED KMEANS CLUSTERING ---
for size in selected_patch_sizes:
    print(f"\n🔍 Running KMeans on {size}x{size} features with silhouette scoring")

    features = all_features[patch_sizes.index(size)][:min_len]  # align patch count

    sil_scores = []
    all_models = []
    all_labels = []

    for k in range(3, 21):
        km = KMeans(n_clusters=k, random_state=42)
        labels = km.fit_predict(features)
        sil = silhouette_score(features, labels)
        sil_scores.append(sil)
        all_models.append(km)
        all_labels.append(labels)

    best_idx = int(np.argmax(sil_scores))
    best_k = best_idx + 3
    best_kmeans = all_models[best_idx]
    best_labels = all_labels[best_idx]
    best_score = sil_scores[best_idx]

    best_kmeans_models[size] = best_kmeans
    cluster_labels_dict[size] = best_labels
    silhouette_scores_per_size[size] = best_score

    # Compute entropy of cluster label distribution
    label_counts = np.bincount(best_labels)
    probs = label_counts / np.sum(label_counts)
    ent = entropy(probs)
    entropy_scores[size] = ent

    print(f"✅ Best k={best_k} | Silhouette={best_score:.3f} | Entropy={ent:.3f}")

    # Plot silhouette curve
    plt.figure(figsize=(6, 3))
    plt.plot(range(3, 21), sil_scores, marker='o')
    plt.title(f'Silhouette Scores for {size}x{size}')
    plt.xlabel('k')
    plt.ylabel('Score')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# --- META-FEATURE COMBINATION ---
print("\n🔗 Combining cluster labels into meta-feature vectors")
meta_features = np.column_stack([cluster_labels_dict[size] for size in selected_patch_sizes])
print(f"Meta-feature shape: {meta_features.shape}")

# --- META-FEATURE DIAGNOSTICS ---

# Correlation Matrix
print("\n📊 Label Correlation Matrix (Pearson):")
corr_matrix = np.corrcoef(meta_features.T)
print(np.round(corr_matrix, 3))

# Entropy of each label column
print("\n🧠 Entropy of Cluster Label Distributions:")
for size in selected_patch_sizes:
    print(f"  - {size}x{size}: Entropy = {entropy_scores[size]:.3f}")

# --- ISOLATION FOREST FOR ANOMALY DETECTION ---
print("\n🌌 Running Isolation Forest on meta-cluster features...")
iso = IsolationForest(contamination=0.05, random_state=42)
anomaly_labels = iso.fit_predict(meta_features)

# Convert -1 to 1 (anomaly), 1 to 0 (normal)
anomaly_scores = -1 * (anomaly_labels - 1) // 2

# Summary
unique_vals, counts = np.unique(anomaly_scores, return_counts=True)
print(f"🧭 Anomaly score distribution: {dict(zip(unique_vals, counts))} (1 = Anomaly)")

# --- T-SNE FOR ANOMALY VISUALIZATION ---
print("\n🧪 Visualizing anomalies in t-SNE space...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
meta_tsne = tsne.fit_transform(meta_features)

plt.figure(figsize=(10, 8))
plt.scatter(meta_tsne[:, 0], meta_tsne[:, 1], c=anomaly_scores, cmap='coolwarm', s=10, alpha=0.7)
plt.title('t-SNE of Meta-Cluster Features (Anomaly Detection)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('Component 2')
plt.colorbar(label='Anomaly Score (1 = Anomaly)')
plt.grid(True)
plt.tight_layout()
plt.show()


🧠 How to Use This Practically

(max Entropy is measured as log(k) and Silhoutte from 0.0-1.0)

    Silhouette: Measures cluster separation and compactness.

    Entropy: Measures cluster balance.

So, ideally:

    ✅ High silhouette and high entropy → good clusters, well-separated, evenly sized.

    ❌ High silhouette, low entropy → strong separation but one cluster dominates.

    ❌ Low silhouette, high entropy → evenly sized clusters but poorly separated.

    ❌ Both low → trash clustering.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

# --- CHECK: Silhouette scores for each selected patch size ---
print("\n📈 Silhouette Scores per Patch Size:")
for size in selected_patch_sizes:
    sil = silhouette_scores_per_size.get(size, None)
    if sil is not None:
        print(f"  - {size}x{size}: Silhouette = {sil:.3f}")
    else:
        print(f"  - {size}x{size}: ❌ Not found")

# --- CHECK: Entropy of cluster label distributions ---
print("\n🧠 Entropy of Cluster Label Distributions:")
for size in selected_patch_sizes:
    labels = cluster_labels_dict[size]
    counts = np.bincount(labels)
    probs = counts / np.sum(counts)
    ent = entropy(probs)
    print(f"  - {size}x{size}: Entropy = {ent:.3f}")

# --- CHECK: Label correlation matrix ---
print("\n🔗 Correlation Matrix Between Cluster Label Sets:")

# Create matrix from all selected label vectors
label_matrix = np.column_stack([cluster_labels_dict[size] for size in selected_patch_sizes])

# Compute correlation matrix
corr_matrix = np.corrcoef(label_matrix.T)

# Display as matrix
print("    Patch Sizes:", selected_patch_sizes)
print(np.round(corr_matrix, 3))

# Optional: visualize the correlation matrix
plt.figure(figsize=(6, 5))
plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(label='Correlation')
plt.xticks(range(len(selected_patch_sizes)), [f'{s}x{s}' for s in selected_patch_sizes])
plt.yticks(range(len(selected_patch_sizes)), [f'{s}x{s}' for s in selected_patch_sizes])
plt.title("Cluster Label Correlation Matrix")
plt.tight_layout()
plt.show()


### Misc.

In [ ]:
# Compare to UMAP (often more stable)
from umap import UMAP
umap_emb = UMAP(n_components=2).fit_transform(patches_pca)

In [ ]:
# Overlay clusters on a CMB map
plt.imshow(cluster_map, cmap='viridis', alpha=0.5)
plt.imshow(cmb_map, cmap='inferno', alpha=0.5)

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
labels = clusterer.fit_predict(patches_pca)

In [ ]:
inertias = []
for k in range(min_clusters, max_clusters+1):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(patches_pca)
    inertias.append(kmeans.inertia_)

plt.plot(range(min_clusters, max_clusters+1), inertias, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
# Check cluster sizes
unique, counts = np.unique(cluster_labels, return_counts=True)
print(dict(zip(unique, counts)))

# Visualize problematic clusters
problem_cluster = 0  # Example
problem_indices = np.where(cluster_labels == problem_cluster)[0]
plt.scatter(patches_tsne[problem_indices, 0],
           patches_tsne[problem_indices, 1])
plt.title(f'Problematic Cluster {problem_cluster}')
plt.show()

### 3. Anomaly Detection

Using Isolation Forest to detect anomalies in the CMB data that might correspond to cosmic strings or other interesting features.


### For Singular Clusters

In [ ]:
# Apply Isolation Forest for anomaly detection
iso_forest = IsolationForest(contamination=0.05, random_state=42)
anomaly_scores = iso_forest.fit_predict(patches_pca)

# Convert to anomaly score (higher = more anomalous)
anomaly_scores = -1 * anomaly_scores  # -1 becomes +1 (anomaly), 1 becomes -1 (normal)
print(f"Anomaly Score Range: {anomaly_scores.min()} to {anomaly_scores.max()}")

# Visualize anomalies in t-SNE space
plt.figure(figsize=(12, 10))
scatter = plt.scatter(patches_tsne[:, 0], patches_tsne[:, 1], c=anomaly_scores, cmap='coolwarm', alpha=0.7, s=10)
plt.colorbar(scatter, label='Anomaly Score')
plt.title('Anomaly Detection in CMB Patches')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

# Map anomaly scores back to the original image
anomaly_map = np.zeros(img_cleaned.shape)
anomaly_count = np.zeros(img_cleaned.shape)

for (i, j), score in zip(positions, anomaly_scores):
    anomaly_map[i:i+patch_size, j:j+patch_size] += score
    anomaly_count[i:i+patch_size, j:j+patch_size] += 1

# Average the anomaly scores where patches overlap
mask = anomaly_count > 0
anomaly_map[mask] /= anomaly_count[mask]

plt.figure(figsize=(15, 10))
plt.imshow(anomaly_map, cmap='coolwarm')
plt.title('Anomaly Map of CMB Data (Potential Cosmic String Candidates)')
plt.colorbar(label='Anomaly Score')
plt.show()


In [ ]:
# Use 8x8 patch size for mapping (or adjust to your preference)
chosen_patch_size = 8
_, positions = extract_patches(img_cleaned, patch_size=chosen_patch_size, stride=stride)

# Make sure anomaly scores are the same length as positions
min_len = min(len(positions), len(anomaly_scores))
positions = positions[:min_len]
anomaly_scores = anomaly_scores[:min_len]

# --- Build anomaly map ---
anomaly_map = np.zeros_like(img_cleaned)
anomaly_count = np.zeros_like(img_cleaned)

for (i, j), score in zip(positions, anomaly_scores):
    anomaly_map[i:i+chosen_patch_size, j:j+chosen_patch_size] += score
    anomaly_count[i:i+chosen_patch_size, j:j+chosen_patch_size] += 1

# Average overlapping patches
mask = anomaly_count > 0
anomaly_map[mask] /= anomaly_count[mask]

# Optional: clip for visualization clarity
vmin, vmax = np.percentile(anomaly_map[mask], [5, 95])

plt.figure(figsize=(15, 10))
plt.imshow(anomaly_map, cmap='coolwarm', vmin=vmin, vmax=vmax)
plt.colorbar(label='Anomaly Score')
plt.title('CMB Anomaly Map (Potential Topological Signatures)')
plt.axis('off')
plt.show()


In [ ]:
# Show top N anomalies
N = 10
top_indices = np.argsort(anomaly_scores)[-N:]

plt.figure(figsize=(15, 3))
for idx, i in enumerate(top_indices):
    patch = scaler.inverse_transform(pca.inverse_transform(combined_features[i]))[:chosen_patch_size**2]
    patch_img = patch.reshape(chosen_patch_size, chosen_patch_size)
    plt.subplot(1, N, idx + 1)
    plt.imshow(patch_img, cmap='inferno')
    plt.title(f'Anomaly {idx+1}')
    plt.axis('off')
plt.suptitle("Top Anomalous Patches (Multi-scale features)", fontsize=16)
plt.tight_layout()
plt.show()


### For Multiple Clusters

In [ ]:
from sklearn.ensemble import IsolationForest

# --- Apply Isolation Forest on combined multi-scale features ---
print("\n🌌 Running Isolation Forest for anomaly detection...")
iso_forest = IsolationForest(contamination=0.05, random_state=42)
anomaly_labels = iso_forest.fit_predict(combined_features)
anomaly_scores = iso_forest.decision_function(combined_features)  # Higher = more normal

# Invert for easier interpretation (higher = more anomalous)
anomaly_scores = -anomaly_scores
print(f"Anomaly score range: {anomaly_scores.min():.4f} to {anomaly_scores.max():.4f}")

# --- Visualize in t-SNE space ---
plt.figure(figsize=(12, 10))
scatter = plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=anomaly_scores, cmap='coolwarm', alpha=0.7, s=10)
plt.colorbar(scatter, label='Anomaly Score')
plt.title('t-SNE Projection with Anomaly Scores')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True)
plt.show()


### Alternative Method

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE

# --- ISOLATION FOREST: Anomaly Detection on Meta-Cluster Features ---
print("\n🌌 Running Isolation Forest on meta-cluster labels...")

# Build meta-feature matrix from cluster label assignments
meta_features = np.column_stack([cluster_labels_dict[size] for size in selected_patch_sizes])
print(f"Meta-feature shape: {meta_features.shape} (samples × {len(selected_patch_sizes)} features)")

# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
iso_labels = iso_forest.fit_predict(meta_features)

# Convert: -1 → 1 (anomaly), 1 → 0 (normal)
anomaly_scores = -1 * (iso_labels - 1) // 2

# --- Summary Stats ---
unique_vals, counts = np.unique(anomaly_scores, return_counts=True)
summary = dict(zip(unique_vals, counts))
print(f"🧭 Anomaly label distribution: {summary} (1 = Anomaly, 0 = Normal)")

# --- OPTIONAL: t-SNE Visualization of Anomaly Detection ---
print("\n🧪 Visualizing anomalies using t-SNE...")

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
meta_tsne = tsne.fit_transform(meta_features)

plt.figure(figsize=(10, 8))
plt.scatter(meta_tsne[:, 0], meta_tsne[:, 1], c=anomaly_scores, cmap='coolwarm', s=12, alpha=0.7)
plt.title('t-SNE of Meta-Cluster Features (Isolation Forest Anomalies)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('Component 2')
plt.colorbar(label='Anomaly Score (1 = Anomaly)')
plt.grid(True)
plt.tight_layout()
plt.show()


### 4. Feature Importance Analysis

Let's analyze which features (principal components) are most important for distinguishing clusters and anomalies.


### For Singular Clusters

In [ ]:
# Analyze the most important principal components
plt.figure(figsize=(12, 6))
plt.bar(range(n_components), pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Importance of Principal Components')
plt.grid(True)
plt.show()

# Analyze the first few principal components
n_display = min(5, n_components)
plt.figure(figsize=(15, 3*n_display))
for i in range(n_display):
    plt.subplot(n_display, 1, i+1)
    component = pca.components_[i].reshape(1, -1)
    component_image = scaler.inverse_transform(component)[0].reshape(patch_size, patch_size)
    plt.imshow(component_image, cmap='coolwarm')
    plt.title(f'Principal Component {i+1}')
    plt.colorbar()
plt.tight_layout()
plt.show()


### For Multiple Clusters

In [ ]:
# --- Bar Plot: Explained Variance per Scale ---
for size in patch_sizes:
    pca = pca_models[size]
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_)
    plt.title(f'Explained Variance by PCA Components — {size}x{size} patches')
    plt.xlabel('Principal Component')
    plt.ylabel('Explained Variance Ratio')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# --- Visualize Top N PCA Components ---
n_display = 5

for size in patch_sizes:
    pca = pca_models[size]
    scaler = scalers[size]
    components = pca.components_
    patch_len = size * size

    print(f"\n🔬 Visualizing top {n_display} PCA components for {size}x{size} patches")

    plt.figure(figsize=(15, 3 * n_display))
    for i in range(min(n_display, components.shape[0])):
        # Get component and inverse-scale it
        component = components[i].reshape(1, -1)
        restored = scaler.inverse_transform(pca.inverse_transform(component))[0]
        patch = restored[:patch_len].reshape(size, size)

        plt.subplot(n_display, 1, i + 1)
        plt.imshow(patch, cmap='coolwarm')
        plt.colorbar()
        plt.title(f'{size}x{size} Patch — Principal Component {i+1}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()


### 5. Correlation with Edge Detection

Let's compare our ML-based anomaly detection with the edge detection performed earlier to see if they identify similar features.


### For Singular Cluster

In [ ]:
# Ensure we have the edge detection results
if 'edges' not in locals():
    edges = ndimage.sobel(img_cleaned)

# Normalize both maps for comparison
edges_norm = (edges - np.min(edges)) / (np.max(edges) - np.min(edges))
anomaly_norm = (anomaly_map - np.min(anomaly_map)) / (np.max(anomaly_map) - np.min(anomaly_map))

# Calculate correlation between edge detection and anomaly detection
valid_mask = ~np.isnan(edges_norm) & ~np.isnan(anomaly_norm)
correlation = np.corrcoef(edges_norm[valid_mask].flatten(), anomaly_norm[valid_mask].flatten())[0, 1]
print(f"Correlation between edge detection and anomaly detection: {correlation:.3f}")

# Visualize the comparison
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(edges_norm, cmap='inferno')
plt.title('Edge Detection')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(anomaly_norm, cmap='coolwarm')
plt.title('Anomaly Detection')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(edges_norm, cmap='inferno', alpha=0.7)
plt.imshow(anomaly_norm, cmap='coolwarm', alpha=0.3)
plt.title('Edge + Anomaly Overlay')
plt.colorbar()

plt.tight_layout()
plt.show()


### For Multiple Clusters

In [ ]:
from scipy import ndimage

# --- ENSURE EDGE DETECTION EXISTS ---
if 'edges' not in locals():
    print("🔍 Computing Sobel edge detection...")
    edges = ndimage.sobel(img_cleaned)

# --- NORMALIZE MAPS FOR COMPARISON ---
def normalize_map(x):
    x = np.nan_to_num(x)
    return (x - np.min(x)) / (np.max(x) - np.min(x) + 1e-8)

edges_norm = normalize_map(edges)
anomaly_norm = normalize_map(anomaly_map)

# --- CREATE VALID MASK ---
valid_mask = (~np.isnan(edges_norm)) & (~np.isnan(anomaly_norm))
edges_flat = edges_norm[valid_mask].flatten()
anomaly_flat = anomaly_norm[valid_mask].flatten()

# --- CORRELATION CALCULATION ---
if edges_flat.size > 0 and anomaly_flat.size > 0:
    correlation = np.corrcoef(edges_flat, anomaly_flat)[0, 1]
    print(f"📈 Correlation between edge detection and anomaly detection: {correlation:.3f}")
else:
    correlation = np.nan
    print("⚠️ No valid pixels for correlation.")

# --- VISUALIZATION ---
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
plt.imshow(edges_norm, cmap='inferno')
plt.title('🟠 Edge Detection (Sobel)')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(anomaly_norm, cmap='coolwarm')
plt.title('🔵 Anomaly Score (IForest)')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.imshow(edges_norm, cmap='inferno', alpha=0.7)
plt.imshow(anomaly_norm, cmap='coolwarm', alpha=0.3)
plt.title('🎯 Edge + Anomaly Overlay')
plt.colorbar()

plt.tight_layout()
plt.show()


In [ ]:
# Create a more informative overlay visualization
plt.figure(figsize=(15, 10))

# Plot the edge detection map with a semi-transparent inferno colormap
plt.imshow(edges_norm, cmap='inferno', alpha=0.7)

# Overlay the anomaly map with a semi-transparent coolwarm colormap
plt.imshow(anomaly_norm, cmap='coolwarm', alpha=0.5)

# Add title and labels
plt.title('Enhanced Edge and Anomaly Overlay')

# Add colorbars for both maps
from mpl_toolkits.axes_grid1 import make_axes_locatable
ax = plt.gca()
divider = make_axes_locatable(ax)

# Add colorbar for edges
cax1 = divider.append_axes("right", size="5%", pad=0.05)
cbar1 = plt.colorbar(plt.cm.ScalarMappable(cmap='inferno'), cax=cax1)
cbar1.set_label('Edge Intensity')

# Add colorbar for anomaly
cax2 = divider.append_axes("right", size="5%", pad=0.3)
cbar2 = plt.colorbar(plt.cm.ScalarMappable(cmap='coolwarm'), cax=cax2)
cbar2.set_label('Anomaly Score')

plt.tight_layout()
plt.show()

# Also show the product of the two maps for comparison
overlap_map = edges_norm * anomaly_norm
plt.figure(figsize=(10, 6))
plt.imshow(overlap_map, cmap='plasma')
plt.title('Overlap Map (Edge × Anomaly)')
plt.colorbar(label='Overlap Intensity')
plt.show()


In [ ]:
# Analyze overlayed elements and identify outliers
print("Analyzing overlayed elements and identifying outliers...")

# Calculate statistics for the overlap map
overlap_mean = np.mean(overlap_map)
overlap_std = np.std(overlap_map)
overlap_median = np.median(overlap_map)
overlap_min = np.min(overlap_map)
overlap_max = np.max(overlap_map)

print(f"Overlap Map Statistics:")
print(f"  Mean: {overlap_mean:.6f}")
print(f"  Median: {overlap_median:.6f}")
print(f"  Standard Deviation: {overlap_std:.6f}")
print(f"  Min: {overlap_min:.6f}")
print(f"  Max: {overlap_max:.6f}")

# Define outliers using standard deviation method (Z-score)
z_threshold = 2.5  # Points with Z-score > 2.5 are considered outliers (covers ~99% of normal distribution)
overlap_z_scores = (overlap_map - overlap_mean) / overlap_std
outliers_mask = np.abs(overlap_z_scores) > z_threshold

# Count outliers
num_outliers = np.sum(outliers_mask)
total_pixels = overlap_map.size
outlier_percentage = (num_outliers / total_pixels) * 100

print(f"\nOutlier Analysis (Z-score > {z_threshold}):")
print(f"  Number of outliers: {num_outliers}")
print(f"  Percentage of outliers: {outlier_percentage:.2f}%")

# Create a visualization of the outliers
plt.figure(figsize=(15, 10))

# Plot the original overlap map
plt.subplot(2, 2, 1)
plt.imshow(overlap_map, cmap='plasma')
plt.title('Original Overlap Map')
plt.colorbar(label='Overlap Intensity')

# Plot the Z-scores
plt.subplot(2, 2, 2)
plt.imshow(overlap_z_scores, cmap='seismic', vmin=-5, vmax=5)
plt.title('Z-scores')
plt.colorbar(label='Z-score')

# Plot the outliers mask
plt.subplot(2, 2, 3)
plt.imshow(outliers_mask, cmap='gray')
plt.title(f'Outliers (Z-score > {z_threshold})')
plt.colorbar(label='Is Outlier')

# Plot the original image with outliers highlighted
plt.subplot(2, 2, 4)
highlighted_map = np.copy(img_cleaned)
highlighted_map = normalize(highlighted_map)  # Normalize for better visualization
highlighted_outliers = np.zeros_like(highlighted_map)
highlighted_outliers[outliers_mask] = 1

plt.imshow(highlighted_map, cmap='gray', alpha=0.7)
plt.imshow(highlighted_outliers, cmap='hot', alpha=0.5)
plt.title('CMB Map with Outliers Highlighted')
plt.colorbar(label='Outlier Intensity')

plt.tight_layout()
plt.show()

# Analyze the characteristics of the outliers
if num_outliers > 0:
    # Extract values of outliers
    outlier_values = overlap_map[outliers_mask]

    # Calculate statistics for outliers
    outlier_mean = np.mean(outlier_values)
    outlier_std = np.std(outlier_values)
    outlier_median = np.median(outlier_values)
    outlier_min = np.min(outlier_values)
    outlier_max = np.max(outlier_values)

    print("\nOutlier Values Statistics:")
    print(f"  Mean: {outlier_mean:.6f}")
    print(f"  Median: {outlier_median:.6f}")
    print(f"  Standard Deviation: {outlier_std:.6f}")
    print(f"  Min: {outlier_min:.6f}")
    print(f"  Max: {outlier_max:.6f}")

    # Histogram of outlier values
    plt.figure(figsize=(10, 6))
    plt.hist(outlier_values, bins=30, alpha=0.7, color='purple')
    plt.title('Distribution of Outlier Values')
    plt.xlabel('Overlap Intensity')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()

    # Compare edge and anomaly values at outlier positions
    edge_values_at_outliers = edges_norm[outliers_mask]
    anomaly_values_at_outliers = anomaly_norm[outliers_mask]

    plt.figure(figsize=(10, 6))
    plt.scatter(edge_values_at_outliers, anomaly_values_at_outliers, alpha=0.5, c=outlier_values, cmap='plasma')
    plt.colorbar(label='Overlap Intensity')
    plt.title('Edge vs Anomaly Values at Outlier Positions')
    plt.xlabel('Edge Detection Value')
    plt.ylabel('Anomaly Detection Value')
    plt.grid(True, alpha=0.3)
    plt.show()


### 6. Wavelet-Based Multi-Scale Analysis for Cosmic String Detection

Wavelets are particularly well-suited for detecting line-like structures such as cosmic strings because they can capture both spatial and frequency information simultaneously. Unlike Fourier transforms, wavelets provide localization in both space and frequency domains, making them ideal for detecting transient features like cosmic strings.


In [ ]:
# Import PyWavelets library
import pywt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage.restoration import denoise_wavelet

# Ensure we have the cleaned CMB image
if 'img_cleaned' not in locals():
    print("Loading and cleaning CMB image...")
    cmb_map = hp.read_map('data/COM_CMB_IQU-smica_2048_R3.00_full.fits', field=0)
    img = hp.mollview(cmb_map, return_projected_map=True, nest=False, title='CMB Temperature Map', cmap='inferno', xsize=2000, hold=True)
    img_cleaned = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)

print(f"Image shape: {img_cleaned.shape}")

# Define wavelet parameters
wavelet = 'cgau8'  # Complex Gaussian wavelet (good for detecting edges and line-like features)
max_level = 5      # Maximum decomposition level

# Perform 2D continuous wavelet transform
coeffs = pywt.wavedec2(img_cleaned, wavelet='db4', level=max_level)

# Extract approximation and detail coefficients
approx = coeffs[0]
details = coeffs[1:]

# Visualize approximation coefficients
plt.figure(figsize=(12, 8))
plt.imshow(approx, cmap='inferno')
plt.title(f'Approximation Coefficients (Level {max_level})')
plt.colorbar(label='Coefficient Value')
plt.show()

# Visualize detail coefficients at each level
plt.figure(figsize=(15, 12))
for i, detail_coeffs in enumerate(details):
    level = i + 1
    # Each detail contains horizontal, vertical, and diagonal coefficients
    horizontal, vertical, diagonal = detail_coeffs

    # Plot horizontal coefficients (sensitive to vertical edges)
    plt.subplot(max_level, 3, 3*i + 1)
    plt.imshow(horizontal, cmap='coolwarm')
    plt.title(f'Horizontal Detail (Level {level})')
    plt.colorbar()

    # Plot vertical coefficients (sensitive to horizontal edges)
    plt.subplot(max_level, 3, 3*i + 2)
    plt.imshow(vertical, cmap='coolwarm')
    plt.title(f'Vertical Detail (Level {level})')
    plt.colorbar()

    # Plot diagonal coefficients
    plt.subplot(max_level, 3, 3*i + 3)
    plt.imshow(diagonal, cmap='coolwarm')
    plt.title(f'Diagonal Detail (Level {level})')
    plt.colorbar()

plt.tight_layout()
plt.show()

# Enhance cosmic string detection by focusing on specific scales
# Cosmic strings are expected to be more prominent at certain scales
enhanced_coeffs = coeffs.copy()

# Modify coefficients to enhance line-like structures
# Boost mid-level details (levels 2-3) where cosmic strings might be more visible
boost_levels = [1, 2]  # 0-indexed
boost_factor = 2.0

for level in boost_levels:
    # Boost horizontal, vertical, and diagonal coefficients
    enhanced_coeffs[level+1] = tuple(coeff * boost_factor for coeff in enhanced_coeffs[level+1])

# Reconstruct image from enhanced coefficients
enhanced_img = pywt.waverec2(enhanced_coeffs, wavelet='db4')

# Ensure reconstructed image has same shape as original
if enhanced_img.shape != img_cleaned.shape:
    enhanced_img = enhanced_img[:img_cleaned.shape[0], :img_cleaned.shape[1]]

# Visualize original vs enhanced image
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.imshow(img_cleaned, cmap='inferno')
plt.title('Original CMB Image')
plt.colorbar(label='Temperature')

plt.subplot(1, 2, 2)
plt.imshow(enhanced_img, cmap='inferno')
plt.title('Wavelet-Enhanced Image (Cosmic String Detection)')
plt.colorbar(label='Temperature')

plt.tight_layout()
plt.show()

# Calculate the difference to highlight enhanced features
difference = enhanced_img - img_cleaned

plt.figure(figsize=(12, 8))
plt.imshow(difference, cmap='coolwarm')
plt.title('Enhanced Features (Potential Cosmic Strings)')
plt.colorbar(label='Difference')
plt.show()

# Apply wavelet-based denoising to better isolate cosmic string candidates
denoised = denoise_wavelet(img_cleaned, method='BayesShrink', mode='soft', 
                          wavelet='db8', wavelet_levels=3)

# Calculate edge map from denoised image
from scipy import ndimage
edges_denoised = ndimage.sobel(denoised)
edges_denoised_norm = normalize(edges_denoised)

# Visualize denoised image and its edges
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.imshow(denoised, cmap='inferno')
plt.title('Wavelet-Denoised CMB Image')
plt.colorbar(label='Temperature')

plt.subplot(1, 2, 2)
plt.imshow(edges_denoised_norm, cmap='inferno')
plt.title('Edge Detection on Denoised Image')
plt.colorbar(label='Edge Intensity')

plt.tight_layout()
plt.show()

# Compare with previous edge detection results
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.imshow(edges_norm, cmap='inferno')
plt.title('Original Edge Detection')
plt.colorbar(label='Edge Intensity')

plt.subplot(1, 2, 2)
plt.imshow(edges_denoised_norm, cmap='inferno')
plt.title('Wavelet-Based Edge Detection')
plt.colorbar(label='Edge Intensity')

plt.tight_layout()
plt.show()

# Calculate correlation between original and wavelet-based edge detection
valid_mask = ~np.isnan(edges_norm) & ~np.isnan(edges_denoised_norm)
correlation = np.corrcoef(edges_norm[valid_mask].flatten(), 
                         edges_denoised_norm[valid_mask].flatten())[0, 1]
print(f"Correlation between original and wavelet-based edge detection: {correlation:.12f}")



🎯 So what’s a "good" correlation?
Correlation (r)	Interpretation	Good for...
0.9	Very similar, low change	Noise removal only
0.7 – 0.9	Minor enhancement	Preserving dominant features
0.4 – 0.7	Moderate enhancement	Revealing subtle structures
< 0.4	Large change	New features revealed, risky if noise amplification

### 7. Directional Analysis of Potential Cosmic Strings

Cosmic strings are expected to have specific directional properties. By analyzing the orientation and direction of detected edges, we can identify patterns that might be characteristic of cosmic strings and distinguish them from other features.


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.feature import hog
from skimage import feature, color, exposure, transform
import cv2

# Ensure we have the edge detection results
if 'edges_norm' not in locals():
    # Normalize edges for visualization
    edges_norm = normalize(edges)

# Use Hough transform to detect lines in the edge map
# First, threshold the edge map to create a binary image
edge_threshold = 0.5
binary_edges = (edges_norm > edge_threshold).astype(np.uint8)

# Apply Hough transform
theta = np.linspace(-np.pi/2, np.pi/2, 180)
h, theta, d = transform.hough_line(binary_edges, theta=theta)

# Visualize the Hough transform
plt.figure(figsize=(15, 10))

plt.subplot(1, 2, 1)
plt.imshow(binary_edges, cmap='gray')
plt.title('Thresholded Edge Map')
plt.colorbar(label='Edge Intensity')

plt.subplot(1, 2, 2)
plt.imshow(np.log1p(h), cmap='inferno', aspect='auto',
           extent=[np.rad2deg(theta[0]), np.rad2deg(theta[-1]), d[-1], d[0]])
plt.title('Hough Transform (Log Scale)')
plt.xlabel('Angle (degrees)')
plt.ylabel('Distance (pixels)')
plt.colorbar(label='Log(votes)')

plt.tight_layout()
plt.show()

# Find peaks in the Hough transform to identify significant lines
hough_threshold = 0.5 * np.max(h)  # Adjust threshold as needed
peaks = feature.peak_local_max(h, min_distance=10, threshold_abs=hough_threshold)

# Extract angles and distances of detected lines
angles = theta[peaks[:, 1]]
distances = d[peaks[:, 0]]

# Visualize detected lines on the original image
plt.figure(figsize=(15, 10))
plt.imshow(img_cleaned, cmap='inferno')
plt.title('Detected Lines (Potential Cosmic Strings)')

for angle, dist in zip(angles, distances):
    # Calculate endpoints of the line
    a = np.cos(angle)
    b = np.sin(angle)
    x0 = a * dist
    y0 = b * dist

    # Calculate line endpoints (extending to image boundaries)
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))

    # Plot the line
    plt.plot([x1, x2], [y1, y2], 'r-', alpha=0.7)

plt.axis('off')
plt.tight_layout()
plt.show()

# Analyze the distribution of line orientations
plt.figure(figsize=(12, 6))
plt.hist(np.rad2deg(angles), bins=36, range=(-90, 90))
plt.title('Distribution of Line Orientations')
plt.xlabel('Angle (degrees)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.show()

# Calculate Histogram of Oriented Gradients (HOG) for directional analysis
# HOG is particularly good at capturing the local orientation of gradients
hog_features, hog_image = hog(
    img_cleaned, 
    orientations=8, 
    pixels_per_cell=(16, 16),
    cells_per_block=(1, 1), 
    visualize=True,
    feature_vector=True
)

# Enhance HOG visualization
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

# Visualize HOG
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
plt.imshow(img_cleaned, cmap='inferno')
plt.title('Original CMB Image')
plt.colorbar(label='Temperature')

plt.subplot(1, 2, 2)
plt.imshow(hog_image_rescaled, cmap='inferno')
plt.title('Histogram of Oriented Gradients (HOG)')
plt.colorbar(label='Gradient Magnitude')

plt.tight_layout()
plt.show()

# Analyze gradient direction using Sobel operators
# Calculate gradients in x and y directions
grad_x = ndimage.sobel(img_cleaned, axis=1)
grad_y = ndimage.sobel(img_cleaned, axis=0)

# Calculate gradient magnitude and direction
grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
grad_direction = np.arctan2(grad_y, grad_x)

# Convert direction to degrees
grad_direction_deg = np.rad2deg(grad_direction)

# Visualize gradient magnitude and direction
plt.figure(figsize=(15, 10))

plt.subplot(1, 2, 1)
plt.imshow(grad_magnitude, cmap='inferno')
plt.title('Gradient Magnitude')
plt.colorbar(label='Magnitude')

plt.subplot(1, 2, 2)
plt.imshow(grad_direction_deg, cmap='hsv')
plt.title('Gradient Direction (degrees)')
plt.colorbar(label='Direction (degrees)')

plt.tight_layout()
plt.show()

# Create a quiver plot to visualize gradient vectors
# Downsample for clarity
step = 20
y, x = np.mgrid[0:img_cleaned.shape[0]:step, 0:img_cleaned.shape[1]:step]
u = grad_x[::step, ::step]
v = grad_y[::step, ::step]

plt.figure(figsize=(15, 10))
plt.imshow(img_cleaned, cmap='inferno', alpha=0.7)
plt.quiver(x, y, u, v, color='white', scale=50, alpha=0.8)
plt.title('Gradient Vector Field (Potential Flow Directions)')
plt.axis('off')
plt.tight_layout()
plt.show()

# Identify regions with consistent gradient direction (potential cosmic strings)
# Calculate the local consistency of gradient directions
from scipy.ndimage import generic_filter

def direction_consistency(x):
    # Convert angles to complex numbers and calculate mean
    z = np.exp(1j * x)
    mean_z = np.mean(z)
    # Return the magnitude of the mean (1 = perfectly aligned, 0 = random)
    return np.abs(mean_z)

# Apply filter to gradient directions with a 5x5 window
consistency = generic_filter(grad_direction, direction_consistency, size=5)

# Visualize direction consistency
plt.figure(figsize=(12, 8))
plt.imshow(consistency, cmap='inferno')
plt.title('Gradient Direction Consistency (Higher = More Aligned)')
plt.colorbar(label='Consistency (0-1)')
plt.tight_layout()
plt.show()

# Combine direction consistency with gradient magnitude to identify potential cosmic strings
cosmic_string_score = consistency * normalize(grad_magnitude)

# Visualize the cosmic string score
plt.figure(figsize=(12, 8))
plt.imshow(cosmic_string_score, cmap='inferno')
plt.title('Cosmic String Score (Direction Consistency × Gradient Magnitude)')
plt.colorbar(label='Score')
plt.tight_layout()
plt.show()

# Compare with previous detection methods
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(edges_norm, cmap='inferno')
plt.title('Edge Detection')
plt.colorbar(label='Edge Intensity')

plt.subplot(2, 2, 2)
plt.imshow(anomaly_norm, cmap='coolwarm')
plt.title('Anomaly Detection')
plt.colorbar(label='Anomaly Score')

plt.subplot(2, 2, 3)
plt.imshow(edges_denoised_norm, cmap='inferno')
plt.title('Wavelet-Based Edge Detection')
plt.colorbar(label='Edge Intensity')

plt.subplot(2, 2, 4)
plt.imshow(cosmic_string_score, cmap='inferno')
plt.title('Directional Analysis Score')
plt.colorbar(label='Score')

plt.tight_layout()
plt.show()


### 8. Statistical Significance Testing of Detected Features

To assess the significance of the features detected by our various methods, we need to compare them against what would be expected from random Gaussian fluctuations in the CMB. This helps distinguish genuine cosmic string candidates from statistical flukes.


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import ndimage
import healpy as hp
from tqdm import tqdm

# Function to generate random Gaussian CMB maps
def generate_random_cmb(nside=2048, lmax=2000, seed=None):
    """Generate a random Gaussian CMB map with the same power spectrum as the observed CMB."""
    # Set random seed for reproducibility if provided
    if seed is not None:
        np.random.seed(seed)

    # Get power spectrum from the real CMB map
    cl = hp.anafast(cmb_map, lmax=lmax)

    # Generate random map with the same power spectrum
    random_map = hp.synfast(cl, nside, lmax=lmax, verbose=False)

    # Project to 2D
    random_img = hp.mollview(random_map, return_projected_map=True, 
                            nest=False, title='Random CMB Map', 
                            cmap='inferno', xsize=2000, hold=True)

    # Clean the map
    random_img_cleaned = np.nan_to_num(random_img, nan=0.0, posinf=0.0, neginf=0.0)

    return random_img_cleaned

# Generate a set of random CMB maps for comparison
n_simulations = 10
print(f"Generating {n_simulations} random CMB simulations...")
random_maps = [generate_random_cmb(seed=i) for i in range(n_simulations)]

# Visualize a few random maps
plt.figure(figsize=(15, 10))
for i in range(min(4, n_simulations)):
    plt.subplot(2, 2, i+1)
    plt.imshow(random_maps[i], cmap='inferno')
    plt.title(f'Random CMB Simulation #{i+1}')
    plt.colorbar(label='Temperature')
plt.tight_layout()
plt.show()

# Function to calculate feature metrics for a given map
def calculate_feature_metrics(img):
    """Calculate various feature metrics for a given map."""
    # Edge detection
    edges = ndimage.sobel(img)
    edges_norm = normalize(edges)

    # Gradient direction analysis
    grad_x = ndimage.sobel(img, axis=1)
    grad_y = ndimage.sobel(img, axis=0)
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    grad_direction = np.arctan2(grad_y, grad_x)

    # Direction consistency (5x5 window)
    def direction_consistency(x):
        z = np.exp(1j * x)
        mean_z = np.mean(z)
        return np.abs(mean_z)

    consistency = generic_filter(grad_direction, direction_consistency, size=5)

    # Cosmic string score
    cosmic_string_score = consistency * normalize(grad_magnitude)

    # Calculate metrics
    metrics = {
        'edge_mean': np.mean(edges_norm),
        'edge_std': np.std(edges_norm),
        'edge_max': np.max(edges_norm),
        'consistency_mean': np.mean(consistency),
        'consistency_std': np.std(consistency),
        'consistency_max': np.max(consistency),
        'cosmic_string_score_mean': np.mean(cosmic_string_score),
        'cosmic_string_score_std': np.std(cosmic_string_score),
        'cosmic_string_score_max': np.max(cosmic_string_score),
    }

    return metrics, edges_norm, cosmic_string_score

# Calculate metrics for the real CMB map
print("Calculating metrics for the real CMB map...")
real_metrics, real_edges_norm, real_cosmic_string_score = calculate_feature_metrics(img_cleaned)

# Calculate metrics for each random map
print("Calculating metrics for random simulations...")
random_metrics_list = []
random_edges_norm_list = []
random_cosmic_string_score_list = []

for i, random_map in enumerate(random_maps):
    print(f"Processing simulation {i+1}/{n_simulations}...")
    metrics, edges_norm, cosmic_string_score = calculate_feature_metrics(random_map)
    random_metrics_list.append(metrics)
    random_edges_norm_list.append(edges_norm)
    random_cosmic_string_score_list.append(cosmic_string_score)

# Compute statistics across random simulations
random_stats = {}
for key in real_metrics.keys():
    values = [metrics[key] for metrics in random_metrics_list]
    random_stats[key] = {
        'mean': np.mean(values),
        'std': np.std(values),
        'min': np.min(values),
        'max': np.max(values)
    }

# Calculate significance (z-scores) for each metric
significance = {}
for key in real_metrics.keys():
    mean = random_stats[key]['mean']
    std = random_stats[key]['std']
    z_score = (real_metrics[key] - mean) / std
    significance[key] = {
        'real_value': real_metrics[key],
        'random_mean': mean,
        'random_std': std,
        'z_score': z_score,
        'p_value': 2 * (1 - stats.norm.cdf(abs(z_score)))  # Two-tailed p-value
    }

# Display significance results
print("\nStatistical Significance of Feature Metrics:")
print("-" * 80)
print(f"{'Metric':<25} {'Real Value':<15} {'Random Mean':<15} {'Random Std':<15} {'Z-Score':<10} {'P-Value':<10}")
print("-" * 80)
for key, values in significance.items():
    print(f"{key:<25} {values['real_value']:<15.6f} {values['random_mean']:<15.6f} {values['random_std']:<15.6f} {values['z_score']:<10.2f} {values['p_value']:<10.6f}")

# Visualize the distribution of metrics across random simulations
plt.figure(figsize=(15, 10))
metrics_to_plot = ['edge_max', 'consistency_max', 'cosmic_string_score_max']
for i, key in enumerate(metrics_to_plot):
    plt.subplot(len(metrics_to_plot), 1, i+1)
    values = [metrics[key] for metrics in random_metrics_list]
    plt.hist(values, bins=20, alpha=0.7)
    plt.axvline(real_metrics[key], color='red', linestyle='--', 
                label=f'Real CMB (z={significance[key]["z_score"]:.2f})')
    plt.title(f'Distribution of {key} across Random Simulations')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.legend()
plt.tight_layout()
plt.show()

# Identify regions with statistically significant features
# Calculate pixel-wise z-scores for cosmic string score
random_cosmic_string_score_mean = np.mean(random_cosmic_string_score_list, axis=0)
random_cosmic_string_score_std = np.std(random_cosmic_string_score_list, axis=0)

# Avoid division by zero
random_cosmic_string_score_std = np.where(random_cosmic_string_score_std > 0, 
                                         random_cosmic_string_score_std, 1e-10)

# Calculate z-score map
z_score_map = (real_cosmic_string_score - random_cosmic_string_score_mean) / random_cosmic_string_score_std

# Create significance mask (e.g., z > 3 is significant at p < 0.003)
significance_threshold = 3.0
significant_features_mask = z_score_map > significance_threshold

# Visualize significant features
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.imshow(real_cosmic_string_score, cmap='inferno')
plt.title('Cosmic String Score (Real CMB)')
plt.colorbar(label='Score')

plt.subplot(2, 2, 2)
plt.imshow(random_cosmic_string_score_mean, cmap='inferno')
plt.title('Mean Cosmic String Score (Random CMB)')
plt.colorbar(label='Score')

plt.subplot(2, 2, 3)
plt.imshow(z_score_map, cmap='coolwarm', vmin=-5, vmax=5)
plt.title('Z-Score Map')
plt.colorbar(label='Z-Score')

plt.subplot(2, 2, 4)
plt.imshow(img_cleaned, cmap='inferno')
plt.imshow(significant_features_mask, cmap='binary', alpha=0.3)
plt.title(f'Statistically Significant Features (Z > {significance_threshold})')
plt.colorbar(label='Is Significant')

plt.tight_layout()
plt.show()

# Count and analyze significant features
num_significant_pixels = np.sum(significant_features_mask)
total_pixels = significant_features_mask.size
significant_percentage = (num_significant_pixels / total_pixels) * 100

print(f"\nSignificant Feature Analysis (Z > {significance_threshold}):")
print(f"  Number of significant pixels: {num_significant_pixels}")
print(f"  Percentage of significant pixels: {significant_percentage:.4f}%")
print(f"  Expected percentage by chance: {100 * (1 - stats.norm.cdf(significance_threshold)):.6f}%")

# If there are significant features, analyze their properties
if num_significant_pixels > 0:
    # Extract properties of significant features
    significant_values = real_cosmic_string_score[significant_features_mask]

    # Calculate statistics
    print("\nProperties of Significant Features:")
    print(f"  Mean value: {np.mean(significant_values):.6f}")
    print(f"  Median value: {np.median(significant_values):.6f}")
    print(f"  Standard deviation: {np.std(significant_values):.6f}")
    print(f"  Min value: {np.min(significant_values):.6f}")
    print(f"  Max value: {np.max(significant_values):.6f}")

    # Visualize distribution of significant feature values
    plt.figure(figsize=(10, 6))
    plt.hist(significant_values, bins=30, alpha=0.7, color='purple')
    plt.title('Distribution of Significant Feature Values')
    plt.xlabel('Cosmic String Score')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()


### 9. Interactive Visualization of Results

Interactive visualizations allow users to dynamically explore the data and gain deeper insights. In this section, we'll create interactive plots to better explore the CMB data and the results of our various analyses.


In [ ]:
# Import necessary libraries for interactive visualization
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np

# Create a function for interactive visualization of different maps
def interactive_map_viewer(maps_dict):
    """
    Create an interactive viewer for multiple maps.

    Parameters:
    -----------
    maps_dict : dict
        Dictionary of maps to visualize, with keys as map names and values as map arrays
    """
    # Create dropdown for map selection
    map_dropdown = widgets.Dropdown(
        options=list(maps_dict.keys()),
        value=list(maps_dict.keys())[0],
        description='Map:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create colormap dropdown
    colormap_dropdown = widgets.Dropdown(
        options=['inferno', 'viridis', 'plasma', 'magma', 'cividis', 'coolwarm', 'RdBu_r', 'jet'],
        value='inferno',
        description='Colormap:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create sliders for min/max values
    min_percentile = widgets.FloatSlider(
        value=1.0,
        min=0.0,
        max=49.0,
        step=1.0,
        description='Min Percentile:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    max_percentile = widgets.FloatSlider(
        value=99.0,
        min=51.0,
        max=100.0,
        step=1.0,
        description='Max Percentile:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create output widget for the plot
    output = widgets.Output()

    # Function to update the plot
    def update_plot(*args):
        with output:
            output.clear_output(wait=True)

            # Get selected map
            selected_map = maps_dict[map_dropdown.value]

            # Calculate percentile values for colormap limits
            vmin = np.percentile(selected_map, min_percentile.value)
            vmax = np.percentile(selected_map, max_percentile.value)

            # Create figure
            fig, ax = plt.subplots(figsize=(12, 8))
            im = ax.imshow(selected_map, cmap=colormap_dropdown.value, vmin=vmin, vmax=vmax)
            ax.set_title(f'{map_dropdown.value}')
            plt.colorbar(im, ax=ax)
            plt.tight_layout()
            plt.show()

    # Connect the widgets to the update function
    map_dropdown.observe(update_plot, names='value')
    colormap_dropdown.observe(update_plot, names='value')
    min_percentile.observe(update_plot, names='value')
    max_percentile.observe(update_plot, names='value')

    # Create the initial plot
    update_plot()

    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([map_dropdown, colormap_dropdown]),
        widgets.HBox([min_percentile, max_percentile]),
        output
    ])

    return ui

# Create a dictionary of maps to visualize
maps_to_visualize = {
    'Original CMB': img_cleaned,
    'Edge Detection': edges_norm,
    'Anomaly Detection': anomaly_norm,
    'Wavelet-Enhanced': enhanced_img,
    'Wavelet-Based Edge Detection': edges_denoised_norm,
    'Cosmic String Score': cosmic_string_score,
    'Z-Score Map': z_score_map,
    'Significant Features': significant_features_mask.astype(float)
}

# Display the interactive map viewer
print("Interactive Map Viewer - Select a map and adjust visualization parameters:")
display(interactive_map_viewer(maps_to_visualize))


In [ ]:
# Create a function for interactive comparison of two maps
def interactive_map_comparison(maps_dict):
    """
    Create an interactive viewer to compare two maps side by side or as an overlay.

    Parameters:
    -----------
    maps_dict : dict
        Dictionary of maps to visualize, with keys as map names and values as map arrays
    """
    # Create dropdowns for map selection
    map1_dropdown = widgets.Dropdown(
        options=list(maps_dict.keys()),
        value=list(maps_dict.keys())[0],
        description='Map 1:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    map2_dropdown = widgets.Dropdown(
        options=list(maps_dict.keys()),
        value=list(maps_dict.keys())[1] if len(maps_dict) > 1 else list(maps_dict.keys())[0],
        description='Map 2:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create colormap dropdowns
    cmap1_dropdown = widgets.Dropdown(
        options=['inferno', 'viridis', 'plasma', 'magma', 'cividis', 'coolwarm', 'RdBu_r', 'jet'],
        value='inferno',
        description='Colormap 1:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    cmap2_dropdown = widgets.Dropdown(
        options=['inferno', 'viridis', 'plasma', 'magma', 'cividis', 'coolwarm', 'RdBu_r', 'jet'],
        value='coolwarm',
        description='Colormap 2:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create radio buttons for display mode
    display_mode = widgets.RadioButtons(
        options=['Side by Side', 'Overlay'],
        value='Side by Side',
        description='Display Mode:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create slider for overlay transparency
    alpha_slider = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.05,
        description='Overlay Transparency:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%'),
        disabled=display_mode.value != 'Overlay'
    )

    # Create output widget for the plot
    output = widgets.Output()

    # Function to update the plot
    def update_plot(*args):
        with output:
            output.clear_output(wait=True)

            # Get selected maps
            map1 = maps_dict[map1_dropdown.value]
            map2 = maps_dict[map2_dropdown.value]

            # Create figure
            if display_mode.value == 'Side by Side':
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

                # Plot map 1
                im1 = ax1.imshow(map1, cmap=cmap1_dropdown.value)
                ax1.set_title(map1_dropdown.value)
                plt.colorbar(im1, ax=ax1)

                # Plot map 2
                im2 = ax2.imshow(map2, cmap=cmap2_dropdown.value)
                ax2.set_title(map2_dropdown.value)
                plt.colorbar(im2, ax=ax2)

            else:  # Overlay mode
                fig, ax = plt.subplots(figsize=(12, 8))

                # Plot map 1 as base
                im1 = ax.imshow(map1, cmap=cmap1_dropdown.value)

                # Overlay map 2 with transparency
                im2 = ax.imshow(map2, cmap=cmap2_dropdown.value, alpha=alpha_slider.value)

                ax.set_title(f'Overlay: {map1_dropdown.value} + {map2_dropdown.value}')

                # Add colorbars
                from mpl_toolkits.axes_grid1 import make_axes_locatable
                divider = make_axes_locatable(ax)

                cax1 = divider.append_axes("right", size="5%", pad=0.05)
                cbar1 = plt.colorbar(im1, cax=cax1)
                cbar1.set_label(map1_dropdown.value)

                cax2 = divider.append_axes("right", size="5%", pad=0.3)
                cbar2 = plt.colorbar(im2, cax=cax2)
                cbar2.set_label(map2_dropdown.value)

            plt.tight_layout()
            plt.show()

    # Function to update alpha slider state
    def update_alpha_state(*args):
        alpha_slider.disabled = display_mode.value != 'Overlay'

    # Connect the widgets to the update functions
    map1_dropdown.observe(update_plot, names='value')
    map2_dropdown.observe(update_plot, names='value')
    cmap1_dropdown.observe(update_plot, names='value')
    cmap2_dropdown.observe(update_plot, names='value')
    display_mode.observe(update_plot, names='value')
    display_mode.observe(update_alpha_state, names='value')
    alpha_slider.observe(update_plot, names='value')

    # Create the initial plot
    update_plot()

    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([map1_dropdown, map2_dropdown]),
        widgets.HBox([cmap1_dropdown, cmap2_dropdown]),
        widgets.HBox([display_mode, alpha_slider]),
        output
    ])

    return ui

# Display the interactive map comparison
print("\nInteractive Map Comparison - Compare two maps side by side or as an overlay:")
display(interactive_map_comparison(maps_to_visualize))


In [ ]:
# Create a function for interactive region exploration
def interactive_region_explorer(base_map, feature_maps):
    """
    Create an interactive viewer to explore specific regions of interest.

    Parameters:
    -----------
    base_map : array
        The base map to display
    feature_maps : dict
        Dictionary of feature maps to overlay, with keys as map names and values as map arrays
    """
    # Create sliders for region selection
    x_center = widgets.IntSlider(
        value=base_map.shape[1] // 2,
        min=0,
        max=base_map.shape[1] - 1,
        step=1,
        description='X Center:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    y_center = widgets.IntSlider(
        value=base_map.shape[0] // 2,
        min=0,
        max=base_map.shape[0] - 1,
        step=1,
        description='Y Center:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    window_size = widgets.IntSlider(
        value=100,
        min=20,
        max=500,
        step=10,
        description='Window Size:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create dropdown for feature map selection
    feature_dropdown = widgets.Dropdown(
        options=list(feature_maps.keys()),
        value=list(feature_maps.keys())[0],
        description='Feature Map:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create slider for overlay transparency
    alpha_slider = widgets.FloatSlider(
        value=0.5,
        min=0.0,
        max=1.0,
        step=0.05,
        description='Overlay Transparency:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # Create output widget for the plot
    output = widgets.Output()

    # Function to update the plot
    def update_plot(*args):
        with output:
            output.clear_output(wait=True)

            # Calculate region boundaries
            half_window = window_size.value // 2
            x_min = max(0, x_center.value - half_window)
            x_max = min(base_map.shape[1], x_center.value + half_window)
            y_min = max(0, y_center.value - half_window)
            y_max = min(base_map.shape[0], y_center.value + half_window)

            # Extract regions from maps
            base_region = base_map[y_min:y_max, x_min:x_max]
            feature_region = feature_maps[feature_dropdown.value][y_min:y_max, x_min:x_max]

            # Create figure
            fig, ax = plt.subplots(figsize=(10, 8))

            # Plot base map
            im1 = ax.imshow(base_region, cmap='inferno')

            # Overlay feature map
            im2 = ax.imshow(feature_region, cmap='coolwarm', alpha=alpha_slider.value)

            ax.set_title(f'Region ({x_min}:{x_max}, {y_min}:{y_max}) - {feature_dropdown.value} Overlay')

            # Add colorbars
            from mpl_toolkits.axes_grid1 import make_axes_locatable
            divider = make_axes_locatable(ax)

            cax1 = divider.append_axes("right", size="5%", pad=0.05)
            cbar1 = plt.colorbar(im1, cax=cax1)
            cbar1.set_label('Base Map')

            cax2 = divider.append_axes("right", size="5%", pad=0.3)
            cbar2 = plt.colorbar(im2, cax=cax2)
            cbar2.set_label(feature_dropdown.value)

            plt.tight_layout()
            plt.show()

            # Print region statistics
            print(f"Region Statistics for {feature_dropdown.value}:")
            print(f"  Mean: {np.mean(feature_region):.6f}")
            print(f"  Median: {np.median(feature_region):.6f}")
            print(f"  Standard Deviation: {np.std(feature_region):.6f}")
            print(f"  Min: {np.min(feature_region):.6f}")
            print(f"  Max: {np.max(feature_region):.6f}")

    # Connect the widgets to the update function
    x_center.observe(update_plot, names='value')
    y_center.observe(update_plot, names='value')
    window_size.observe(update_plot, names='value')
    feature_dropdown.observe(update_plot, names='value')
    alpha_slider.observe(update_plot, names='value')

    # Create the initial plot
    update_plot()

    # Create the UI layout
    ui = widgets.VBox([
        widgets.HBox([x_center, y_center]),
        widgets.HBox([window_size, feature_dropdown]),
        widgets.HBox([alpha_slider]),
        output
    ])

    return ui

# Display the interactive region explorer
print("\nInteractive Region Explorer - Zoom in on specific regions of interest:")
display(interactive_region_explorer(img_cleaned, maps_to_visualize))


### 10. Large-Scale Structure Analysis in the CMB

While previous sections focused on detecting small-scale features like cosmic strings, this section explores large-scale structures in the CMB. Large-scale structures correspond to low multipoles (ℓ < 30) and can provide insights into the early universe, inflation, and potential anomalies.


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from matplotlib.ticker import ScalarFormatter
import scipy.stats as stats
from scipy.special import legendre

# Ensure we have the CMB map loaded
if 'cmb_map' not in locals():
    cmb_map = hp.read_map('data/COM_CMB_IQU-smica_2048_R3.00_full.fits', field=0)
    nside = hp.get_nside(cmb_map)
    npix = hp.nside2npix(nside)

# Calculate power spectrum
cl = hp.anafast(cmb_map, lmax=100)
ell = np.arange(len(cl))

# Focus on large scales (low multipoles)
lmax_large = 30  # Define large scales as ℓ < 30

# Plot the power spectrum for large scales
plt.figure(figsize=(12, 8))
plt.plot(ell[1:lmax_large+1], cl[1:lmax_large+1], 'o-', color='darkblue', lw=2)
plt.title('CMB Power Spectrum at Large Scales (Low Multipoles)')
plt.xlabel('Multipole ℓ')
plt.ylabel('C$_\\ell$ [μK$^2$]')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

# Calculate the theoretical ΛCDM power spectrum for comparison
# This is a simplified model - in practice, you would use CAMB or CLASS
def simple_lambda_cdm(ell, A_s=2.1e-9, n_s=0.965):
    """Simple power-law primordial power spectrum with ΛCDM transfer function."""
    # This is a very simplified approximation
    ell = np.asarray(ell)
    ell[ell == 0] = 1  # Avoid division by zero

    # Power-law spectrum with Sachs-Wolfe effect
    cl_theory = A_s * (ell / 80.0) ** (n_s - 1.0) * 2 * np.pi / (ell * (ell + 1))

    # Scale to match observed spectrum approximately
    cl_theory *= 1e10
    return cl_theory

# Calculate theoretical spectrum
cl_theory = simple_lambda_cdm(ell[1:lmax_large+1])

# Plot observed vs theoretical
plt.figure(figsize=(12, 8))
plt.plot(ell[1:lmax_large+1], cl[1:lmax_large+1], 'o-', color='darkblue', lw=2, label='Observed')
plt.plot(ell[1:lmax_large+1], cl_theory, '--', color='red', lw=2, label='ΛCDM (simplified)')
plt.title('CMB Power Spectrum: Observed vs ΛCDM Model')
plt.xlabel('Multipole ℓ')
plt.ylabel('C$_\\ell$ [μK$^2$]')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.show()

# Calculate residuals
residuals = cl[1:lmax_large+1] - cl_theory
percent_diff = 100 * residuals / cl_theory

# Plot residuals
plt.figure(figsize=(12, 6))
plt.bar(ell[1:lmax_large+1], percent_diff, color='purple', alpha=0.7)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.title('Residuals: Observed - ΛCDM Model')
plt.xlabel('Multipole ℓ')
plt.ylabel('Difference (%)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 10.1 Spherical Harmonic Analysis of Large-Scale Structures

Spherical harmonics provide a natural basis for analyzing the CMB, as they capture the angular patterns on the sky. Low-order spherical harmonics correspond to large-scale structures.


In [ ]:
# Visualize the low-order spherical harmonics maps
plt.figure(figsize=(15, 12))

# Plot the first few multipoles (ℓ = 1, 2, 3, 4)
for i, l in enumerate([1, 2, 3, 4]):
    # Get the spherical harmonic coefficients for this multipole
    alm = hp.map2alm(cmb_map, lmax=l)

    # Set all coefficients to zero except for the current multipole
    alm_filtered = np.zeros_like(alm, dtype=complex)

    # Find indices corresponding to the current multipole
    idx = hp.Alm.getidx(l, l, np.arange(0, l+1))
    alm_filtered[idx] = alm[idx]

    # Convert back to map
    map_filtered = hp.alm2map(alm_filtered, nside)

    # Plot
    plt.subplot(2, 2, i+1)
    hp.mollview(map_filtered, title=f'ℓ = {l} (Large-Scale Structure)', hold=True, cmap='coolwarm')

plt.tight_layout()
plt.show()

# Calculate the power in each multipole
power_per_l = np.zeros(lmax_large+1)
for l in range(1, lmax_large+1):
    # Get the spherical harmonic coefficients for this multipole
    alm = hp.map2alm(cmb_map, lmax=l)

    # Set all coefficients to zero except for the current multipole
    alm_filtered = np.zeros_like(alm, dtype=complex)

    # Find indices corresponding to the current multipole
    idx = hp.Alm.getidx(l, l, np.arange(0, l+1))
    alm_filtered[idx] = alm[idx]

    # Convert back to map
    map_filtered = hp.alm2map(alm_filtered, nside)

    # Calculate power (variance of the map)
    power_per_l[l] = np.var(map_filtered)

# Plot power per multipole
plt.figure(figsize=(12, 6))
plt.bar(range(1, lmax_large+1), power_per_l[1:], color='teal', alpha=0.7)
plt.title('Power in Each Multipole (Large-Scale Structures)')
plt.xlabel('Multipole ℓ')
plt.ylabel('Power (Variance)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 10.2 Angular Correlation Function Analysis

The angular correlation function measures how temperature fluctuations are correlated at different angular separations. It's particularly useful for studying large-scale correlations in the CMB.


In [ ]:
# Calculate the angular correlation function
def angular_correlation(map_data, nside, n_theta=100):
    """Calculate the angular correlation function C(θ)."""
    # Generate random points on the sphere
    npix = hp.nside2npix(nside)
    n_points = 10000  # Number of random points to use

    # Generate random pixel indices
    pixels = np.random.choice(npix, size=n_points, replace=False)

    # Get the values at these pixels
    values = map_data[pixels]

    # Get the coordinates of these pixels
    theta, phi = hp.pix2ang(nside, pixels)

    # Convert to 3D vectors
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    vectors = np.column_stack((x, y, z))

    # Calculate all pairwise dot products
    dot_products = np.dot(vectors, vectors.T)

    # Clip to valid range for arccos
    dot_products = np.clip(dot_products, -1.0, 1.0)

    # Convert to angles in degrees
    angles = np.arccos(dot_products) * 180.0 / np.pi

    # Calculate all pairwise products of values
    value_products = np.outer(values, values)

    # Create angle bins
    theta_bins = np.linspace(0, 180, n_theta+1)
    theta_centers = 0.5 * (theta_bins[1:] + theta_bins[:-1])

    # Calculate correlation function
    correlation = np.zeros(n_theta)
    counts = np.zeros(n_theta)

    # Bin the angles and calculate mean product in each bin
    for i in range(n_points):
        for j in range(i+1, n_points):
            angle = angles[i, j]
            bin_idx = np.digitize(angle, theta_bins) - 1
            if 0 <= bin_idx < n_theta:
                correlation[bin_idx] += value_products[i, j]
                counts[bin_idx] += 1

    # Normalize by counts
    mask = counts > 0
    correlation[mask] /= counts[mask]

    return theta_centers, correlation

# Calculate the angular correlation function
theta, corr = angular_correlation(cmb_map, nside, n_theta=50)

# Plot the angular correlation function
plt.figure(figsize=(12, 8))
plt.plot(theta, corr, 'o-', color='darkblue', lw=2)
plt.title('CMB Angular Correlation Function')
plt.xlabel('Angular Separation θ (degrees)')
plt.ylabel('C(θ) [μK$^2$]')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Calculate the theoretical correlation function
# This is a simplified model using Legendre polynomials
def theoretical_correlation(theta_deg, cl):
    """Calculate theoretical correlation function from power spectrum."""
    # Convert to radians
    theta_rad = theta_deg * np.pi / 180.0

    # Initialize correlation function
    corr_theory = np.zeros_like(theta_rad)

    # Sum over multipoles
    lmax = len(cl) - 1
    for l in range(1, lmax+1):
        # Legendre polynomial
        leg = legendre(l)(np.cos(theta_rad))

        # Add contribution from this multipole
        corr_theory += (2*l + 1) * cl[l] * leg / (4 * np.pi)

    return corr_theory

# Calculate theoretical correlation function
corr_theory = theoretical_correlation(theta, cl)

# Plot observed vs theoretical
plt.figure(figsize=(12, 8))
plt.plot(theta, corr, 'o-', color='darkblue', lw=2, label='Observed')
plt.plot(theta, corr_theory, '--', color='red', lw=2, label='Theory')
plt.title('CMB Angular Correlation Function: Observed vs Theory')
plt.xlabel('Angular Separation θ (degrees)')
plt.ylabel('C(θ) [μK$^2$]')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()


### 10.3 Topological Analysis of Large-Scale Structures

Topology provides a way to characterize the morphology of CMB temperature fluctuations. Minkowski functionals are particularly useful for quantifying the topology of the CMB and can reveal non-Gaussian features.


In [ ]:
# Implement a simplified version of Minkowski functionals
def minkowski_functionals(map_data, thresholds):
    """
    Calculate simplified Minkowski functionals for a 2D map.

    Parameters:
    -----------
    map_data : 2D array
        The map data
    thresholds : array
        Threshold values to calculate functionals at

    Returns:
    --------
    v0, v1, v2 : arrays
        The three Minkowski functionals at each threshold
    """
    # Initialize functionals
    n_thresh = len(thresholds)
    v0 = np.zeros(n_thresh)  # Area
    v1 = np.zeros(n_thresh)  # Perimeter
    v2 = np.zeros(n_thresh)  # Euler characteristic

    # Calculate functionals at each threshold
    for i, thresh in enumerate(thresholds):
        # Create binary map
        binary_map = (map_data > thresh).astype(int)

        # V0: Area (fraction of pixels above threshold)
        v0[i] = np.mean(binary_map)

        # V1: Perimeter (simplified using edge detection)
        edges = ndimage.sobel(binary_map)
        v1[i] = np.sum(edges > 0) / binary_map.size

        # V2: Euler characteristic (simplified using structure count)
        # Label connected structures
        labeled, num_features = ndimage.label(binary_map)

        # Count holes using inverted map
        labeled_inv, num_holes = ndimage.label(1 - binary_map)

        # Euler characteristic = structures - holes
        v2[i] = (num_features - num_holes) / binary_map.size

    return v0, v1, v2

# Convert HEALPix map to 2D for topological analysis
if 'img_cleaned' not in locals():
    img = hp.mollview(cmb_map, return_projected_map=True, nest=False, title='CMB Temperature Map', cmap='inferno', xsize=2000, hold=True)
    img_cleaned = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)

# Define thresholds based on standard deviations
sigma = np.std(img_cleaned[~np.isnan(img_cleaned)])
mean = np.mean(img_cleaned[~np.isnan(img_cleaned)])
thresholds = np.linspace(mean - 3*sigma, mean + 3*sigma, 25)

# Calculate Minkowski functionals
v0, v1, v2 = minkowski_functionals(img_cleaned, thresholds)

# Plot Minkowski functionals
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(thresholds, v0, 'o-', color='darkblue', lw=2)
plt.title('V0: Area Fraction')
plt.xlabel('Threshold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(thresholds, v1, 'o-', color='darkred', lw=2)
plt.title('V1: Perimeter')
plt.xlabel('Threshold')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(thresholds, v2, 'o-', color='darkgreen', lw=2)
plt.title('V2: Euler Characteristic')
plt.xlabel('Threshold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Generate Gaussian random field for comparison
np.random.seed(42)
gaussian_map = np.random.normal(mean, sigma, img_cleaned.shape)

# Calculate Minkowski functionals for Gaussian field
v0_gauss, v1_gauss, v2_gauss = minkowski_functionals(gaussian_map, thresholds)

# Plot comparison with Gaussian random field
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(thresholds, v0, 'o-', color='darkblue', lw=2, label='CMB')
plt.plot(thresholds, v0_gauss, '--', color='gray', lw=2, label='Gaussian')
plt.title('V0: Area Fraction')
plt.xlabel('Threshold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(thresholds, v1, 'o-', color='darkred', lw=2, label='CMB')
plt.plot(thresholds, v1_gauss, '--', color='gray', lw=2, label='Gaussian')
plt.title('V1: Perimeter')
plt.xlabel('Threshold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(thresholds, v2, 'o-', color='darkgreen', lw=2, label='CMB')
plt.plot(thresholds, v2_gauss, '--', color='gray', lw=2, label='Gaussian')
plt.title('V2: Euler Characteristic')
plt.xlabel('Threshold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()


### 10.4 Machine Learning for Large-Scale Structure Detection

Machine learning can help identify patterns and anomalies in large-scale CMB structures that might be difficult to detect with traditional methods.


In [ ]:
# Import necessary libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Extract large-scale features from the CMB map
def extract_large_scale_features(cmb_map, lmax=30):
    """Extract large-scale features from CMB map using spherical harmonics."""
    # Get spherical harmonic coefficients
    alm = hp.map2alm(cmb_map, lmax=lmax)

    # Convert to power spectrum
    cl = hp.alm2cl(alm)

    # Get real and imaginary parts of alm coefficients
    alm_real = np.real(alm)
    alm_imag = np.imag(alm)

    # Combine into feature vector
    features = np.concatenate([cl, alm_real, alm_imag])

    return features

# Extract features
lmax_large = 30
large_scale_features = extract_large_scale_features(cmb_map, lmax=lmax_large)

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(large_scale_features.reshape(1, -1))

# Generate a set of random CMB maps for comparison
n_simulations = 10000
print(f"Generating {n_simulations} random CMB simulations for large-scale analysis...")

# Function to generate random CMB maps
def generate_random_cmb(nside=2048, lmax=30, seed=None):
    """Generate a random Gaussian CMB map with the same power spectrum as the observed CMB."""
    # Set random seed for reproducibility if provided
    if seed is not None:
        np.random.seed(seed)

    # Get power spectrum from the real CMB map
    cl = hp.anafast(cmb_map, lmax=lmax)

    # Generate random map with the same power spectrum
    random_map = hp.synfast(cl, nside, lmax=lmax4)
    print("Random CMB map generated.")

    return random_map

# Generate random maps and extract features
random_features = []
for i in range(n_simulations):
    print(f"Generating random CMB map {i+1}/{n_simulations}...")
    random_map = generate_random_cmb(nside=nside, lmax=lmax_large, seed=i)
    features = extract_large_scale_features(random_map, lmax=lmax_large)
    random_features.append(features)

# Convert to array and standardize
random_features = np.array(random_features)
random_features_scaled = scaler.transform(random_features)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=10)
random_features_pca = pca.fit_transform(random_features_scaled)
real_feature_pca = pca.transform(features_scaled)

# Plot the first two principal components
plt.figure(figsize=(10, 8))
plt.scatter(random_features_pca[:, 0], random_features_pca[:, 1], alpha=0.7, label='Random Simulations')
plt.scatter(real_feature_pca[0, 0], real_feature_pca[0, 1], color='red', s=100, marker='*', label='Observed CMB')
plt.title('PCA of Large-Scale CMB Features')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Apply Isolation Forest for anomaly detection
iso = IsolationForest(contamination=0.05, random_state=42)
iso.fit(random_features_pca)

# Calculate anomaly score for real CMB
anomaly_score = -iso.score_samples(real_feature_pca)[0]
print(f"Anomaly score for observed CMB: {anomaly_score:.4f}")

# Calculate anomaly scores for random simulations
random_scores = -iso.score_samples(random_features_pca)

# Plot histogram of anomaly scores
plt.figure(figsize=(10, 6))
plt.hist(random_scores, bins=20, alpha=0.7, color='blue', label='Random Simulations')
plt.axvline(anomaly_score, color='red', linestyle='--', linewidth=2, label='Observed CMB')
plt.title('Anomaly Scores for Large-Scale CMB Features')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Calculate p-value
p_value = np.mean(random_scores >= anomaly_score)
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("The observed CMB shows significant anomalies in large-scale structure compared to random simulations.")
else:
    print("The observed CMB is consistent with random simulations at large scales.")


### 11. Conclusion

We've applied several machine learning and signal processing techniques to analyze the CMB data:

1. **Feature Extraction**: Extracted patches from the CMB map and reduced dimensionality with PCA
2. **Clustering**: Identified distinct patterns in the CMB data using K-means
3. **Anomaly Detection**: Used Isolation Forest to find unusual patterns that might correspond to cosmic strings
4. **Feature Importance**: Analyzed which principal components are most significant
5. **Correlation Analysis**: Compared ML-based anomaly detection with traditional edge detection
6. **Wavelet Analysis**: Applied wavelet transforms to detect multi-scale features and enhance cosmic string detection
7. **Directional Analysis**: Analyzed the orientation and direction of detected edges to identify patterns characteristic of cosmic strings
8. **Statistical Significance Testing**: Compared detected features against random simulations to assess their significance
9. **Interactive Visualization**: Created interactive tools to explore the data and results dynamically
10. **Large-Scale Structure Analysis**: Analyzed the CMB at large angular scales using power spectrum analysis, spherical harmonics, angular correlation functions, topological analysis, and machine learning

These techniques provide complementary views of the CMB data and can help identify potential cosmic string candidates or other interesting features that might not be apparent through traditional analysis methods.

The correlation between edge detection and anomaly detection suggests that ML methods can identify similar structures to traditional methods, but may also reveal additional patterns not captured by edge detection alone.

Wavelet analysis provides a powerful tool for multi-scale feature detection, allowing us to enhance potential cosmic string signatures at specific scales while suppressing noise at other scales.

Directional analysis helps identify regions with consistent gradient directions, which is a characteristic property of cosmic strings. By combining direction consistency with gradient magnitude, we can create a cosmic string score that highlights potential cosmic string candidates.

Statistical significance testing allows us to distinguish genuine cosmic string candidates from statistical flukes by comparing our detected features against what would be expected from random Gaussian fluctuations in the CMB. This provides a rigorous framework for assessing the reality of potential cosmic string detections.

Interactive visualization tools enable researchers to dynamically explore the data and results, facilitating deeper insights and more effective communication of findings. These tools allow for real-time adjustment of visualization parameters, comparison of different analysis methods, and detailed examination of regions of interest.

Our large-scale structure analysis revealed patterns in the CMB at large angular scales, which correspond to the earliest observable epochs of the universe. The power spectrum analysis, spherical harmonic decomposition, angular correlation function, and topological analysis all provide complementary views of these large-scale structures. Machine learning techniques helped identify potential anomalies in these large-scale patterns compared to what would be expected from standard cosmological models.
